# IBK 준법감시팀 유효기간 확인 도구

## 주요 기능 상세 설명

### 1. 다양한 문서 형식 지원
다음 파일 형식을 자동으로 인식하고 처리합니다:
- **문서 파일**: DOC, DOCX (Microsoft Word)
- **스프레드시트**: XLS, XLSX (Microsoft Excel)
- **프레젠테이션**: PPT, PPTX (Microsoft PowerPoint)
- **한글 문서**: HWP (한글과컴퓨터 한글)
- **PDF 문서**: PDF
- **텍스트 파일**: TXT, CSV

각 파일 형식에 맞는 라이브러리(python-docx, openpyxl, python-pptx, olefile, PyPDF2 등)를 사용하여 텍스트를 추출합니다.

### 2. 정교한 준법감시 정보 추출

#### 2.1. 파일명 기반 추출
파일명에서 준법감시 관련 정보를 추출하는 패턴:
- `제2024-4806호(2024.08.21) 유효기간(2025.08.20).hwp` 형식
- `준법감시-2024-123_유효기간_20250820.docx` 형식
- `심의필-2024-456_만료일_2025년08월20일.pptx` 형식

#### 2.2. 문서 내용 기반 추출
문서 내용에서 추출할 수 있는 패턴:
- 준법감시필 번호: `준법감시-2024-123`, `제2024-4806호`, `심의필-2024-456`
- 유효기간 표현: `유효기간: 2025.08.20`, `2025년 8월 20일까지 유효`, `만료일: 2025-08-20`

### 3. 단계적 분석 프로세스

#### 3.1. 실제 구현된 분석 과정
1. **파일 수집**: 파일명에 키워드가 포함된 문서 파일 식별
2. **파일명 분석**: 파일명에서 유효기간 및 준법감시필 번호 추출
3. **문서 내용 분석**: 파일 내용에서 정규식 패턴 매칭을 통한 정보 추출
4. **결과 통합**: 파일명과 내용에서 추출한 정보 통합 및 우선순위 적용

#### 3.2. N-gram 기반 텍스트 분석
- 코드에서는 `AIEnhancedComplianceExtractor` 클래스에서 단순화된 N-gram 분석 구현
- 2-gram, 3-gram, 4-gram을 사용하여 텍스트의 관련성 점수 계산
- 설명된 완전한 자카드 유사도 계산은 `EnhancedTextComparison` 클래스에 구현되어 있으나, 기본 파이프라인에서는 제한적으로 활용됨

### 4. 성능 최적화 기법

#### 4.1. 병렬 처리
- `ThreadPoolExecutor`를 사용한 다중 스레드 병렬 처리
- 파일 검색 및 분석 단계에서 병렬화 적용
- 배치 처리 방식으로 메모리 사용량 관리

#### 4.2. 선택적 문서 처리
- 우선순위 파일 먼저 처리 (키워드 기반)
- 대용량 파일(20MB 초과) 분석 생략 옵션
- 배치 단위 처리로 자원 관리

#### 4.3. 청크 단위 처리
- 대용량 문서의 경우 `create_semantic_chunks()` 함수를 통해 청크로 분할 처리
- 관련성 높은 섹션 선별적 분석으로 효율성 향상
- `score_chunks_by_relevance()` 함수로 관련성 높은 청크 선별

### 5. 유효기간 상태 분석 로직

#### 5.1. 상태 분류 체계
- **만료됨**: 유효기간이 현재 날짜보다 이전인 문서
- **30일 이내 만료**: 유효기간이 현재로부터 30일 이내인 문서
- **유효함**: 유효기간이 현재로부터 30일 이상 남은 문서 
- **상태 불명**: 유효기간을 추출할 수 없는 문서

#### 5.2. 정보 신뢰도
- 파일명에서 추출한 정보를 우선적으로 활용
- 파일 내용에서 발견된 정보는 보조적으로 활용
- 두 정보가 일치할 경우 신뢰도 향상

### 6. 유사 문서 기능

#### 6.1. 제한적 유사도 분석
- `EnhancedTextComparison` 클래스에 유사도 계산 알고리즘 구현
- N-gram 및 해시 기반 비교 방식 지원
- 메인 파이프라인에서는 선택적으로 활성화됨

#### 6.2. 유사도 기반 기능
- `SimilarityBasedInfoPropagation` 클래스를 통해 유사 문서 간 정보 전파 구현
- 유사도가 높은 문서 관계 식별
- 유효기간 정보가 없는 문서에 유사 문서의 정보 전파 가능

### 7. 오류 처리 및 예외 상황 관리

#### 7.1. 오류 날짜 필터링
- 알려진 오류 날짜 목록을 통한 필터링 (`ERROR_DATES` 변수)
- 2020-12-31, 2024-11-30 등 오류로 추출되는 날짜 자동 제외

#### 7.2. 유효성 검증
- 날짜 형식 및 범위 유효성 검증 (2000~2100년, 1~12월, 1~31일)
- 파일명과 내용에서 추출한 정보의 신뢰도 비교
- 정규식 매칭 오류에 대한 예외 처리

#### 7.3. 파일 처리 오류 대응
- 파일 읽기 오류 발생 시 해당 파일 스킵
- 대용량 파일 처리 중 메모리 오류 방지를 위한 한계 설정
- 배치 처리 방식으로 중간 결과 주기적 저장

### 8. 결과 시각화 및 보고서 생성

#### 8.1. Excel 보고서 자동 포맷팅
- 열 너비 자동 조정
- 헤더 행 스타일 설정 (굵은 글꼴, 배경색)
- 상태별 조건부 서식 적용:
  - 만료됨: 밝은 빨강 (#FFC7CE)
  - 30일 이내 만료: 밝은 노랑 (#FFEB9C)
  - 유효함: 밝은 녹색 (#C6EFCE)
  - 상태 불명: 회색 (#DDDDDD)

#### 8.2. 분석 통계 제공
- 상태별 문서 수 및 비율
- 정보 출처 통계 (파일명에서만 발견, 파일 내용에서만 발견, 양쪽 모두, 어디에서도 없음)
- 오류 날짜 포함 문서 목록

### 9. 확장 가능성

#### 9.1. AI 기반 분석 선택적 활성화
- `AIEnhancedComplianceExtractor` 클래스에 API 연동 구조 마련
- 실제 구현에서는 API 호출 비활성화 (주석 처리)
- 필요 시 외부 AI API 연동 가능한 구조

#### 9.2. 배치 처리 및 자동화
- 전체 프로세스 배치 처리 지원
- 분석 중간 결과 저장 기능
- Windows 작업 스케줄러나 Linux cron으로 연동 가능한 구조

## 코드 구현 상세 설명

### 다양한 문서 형식 지원 (셀 #3)

```python
def read_file_content(file_path):
    """다양한 파일 형식에서 텍스트 추출"""
    content = ""
    ext = file_path.suffix.lower()
    
    try:
        # 텍스트 파일
        if ext == '.txt':
            # UTF-8 및 CP949 인코딩 처리
            # ...
        
        # Word 문서
        elif ext == '.docx':
            import docx
            # 문단 추출
            # ...
        
        # Excel 파일
        elif ext in ['.xlsx', '.xls']:
            # 시트 및 셀 데이터 처리
            # ...
        
        # PowerPoint 파일
        elif ext == '.pptx':
            # 슬라이드 텍스트 추출
            # ...
        
        # HWP 파일
        elif ext == '.hwp':
            # olefile로 미리보기 텍스트 추출
            # ...
                
        # PDF 파일
        elif ext == '.pdf':
            # 페이지별 텍스트 추출
            # ...

        return content
    except Exception as e:
        log_debug(f"파일 {file_path.name} 읽기 오류: {str(e)}")
        return ""
```

### 준법감시 정보 추출 (셀 #2 및 셀 #4)

파일명에서 유효기간 추출:

```python
def extract_expiry_from_filename_improved(filename):
    """파일명에서 유효기간 정보를 추출 - 개선된 버전"""
    # 다양한 정규식 패턴으로 유효기간 추출
    ibk_patterns = [
        r'유효기간\(([0-9]{4})[\\.\\-]([0-9]{2})[\\.\\-]([0-9]{2})\)',  # 유효기간(2025.08.20)
        r'유효기간\(([0-9]{4})([0-9]{2})([0-9]{2})\)',  # 유효기간(20250820)
        # 여러 패턴 등록...
    ]
    
    # 패턴 검색 및 날짜 검증
    # ...
```

문서 내용에서 정보 추출:

```python
def extract_with_regex(self, text):
    """정규식으로 준법감시 정보 추출"""
    # 준법감시필 번호 추출 패턴
    compliance_patterns = [
        r'제(\d{4})-(\d+)호',  # 제2024-4806호
        # ...
    ]
    
    # 유효기간 추출 패턴
    expiry_patterns = [
        r'유효기간[^0-9]*(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?',  # 유효기간: 2025.08.20
        # ...
    ]
    # ...
```

### 청크 단위 처리 (셀 #4)

```python
def create_semantic_chunks(self, text):
    """의미론적 청크 생성 - 문장/단락 단위 분할"""
    # 단락 기반 분할
    paragraphs = re.split(r'\n\s*\n', text)
    
    # 청크 생성 로직
    # ...
    
    # 청크 중첩 처리로 맥락 유지
    # ...
```

### 유효기간 분석 (셀 #7)

```python
class ExpiryAnalyzer:
    def __init__(self):
        self.today = datetime.now()
        
    def predict_expiry_date(self, dates, approval_numbers, validity_sentences):
        """다양한 정보를 종합하여 유효기간 예측"""
        potential_expiry_dates = []
        
        # 1. 명시적 유효기간 또는 만료일이 있는 경우
        # 2. 유효기간 문장에 포함된 날짜 활용
        # 3. 준법감시/심의필 번호 기준 추정
        # 4. 일반 날짜 중 미래 날짜 고려
        # ...
```

### 유사 문서 분석 (셀 #6)

```python
class EnhancedTextComparison:
    def __init__(self, min_ngram_size=3, max_ngram_size=5, threshold=0.6):
        self.min_ngram_size = min_ngram_size
        self.max_ngram_size = max_ngram_size
        self.threshold = threshold
    
    def calculate_similarity(self, text1, text2):
        """두 텍스트 간 유사도 계산 (지문 기반)"""
        fingerprints1 = self._create_fingerprints(text1)
        fingerprints2 = self._create_fingerprints(text2)
        
        return self._calculate_jaccard_similarity(fingerprints1, fingerprints2)
    
    # ...
```

### Excel 보고서 생성 (셀 #5)

```python
def save_formatted_excel(df, excel_file):
    """결과를 서식이 적용된 Excel로 저장"""
    # ...
    # 열 너비 자동 조정
    for i, column in enumerate(df.columns):
        # 열 너비 계산 로직
        # ...
        
    # 헤더 행 스타일 설정
    header_font = Font(bold=True)
    header_fill = PatternFill(start_color='E6E6E6', end_color='E6E6E6', fill_type='solid')
    
    # 데이터 행에 조건부 서식 설정
    status_colors = {
        '만료됨': 'FFC7CE',  # 밝은 빨강
        '30일 이내 만료': 'FFEB9C',  # 밝은 노랑
        '유효함': 'C6EFCE',  # 밝은 녹색
        '상태 불명': 'DDDDDD'  # 회색
    }
    # ...
```

### 배치 처리 (셀 #11)

```python
def process_documents_in_batches(file_paths, batch_size=50, max_workers=8):
    """파일 배치 처리 및 병렬화 - 메모리 효율성 개선"""
    # 배치로 나누기
    batches = [file_paths[i:i+batch_size] for i in range(0, len(file_paths), batch_size)]
    
    # 배치별 병렬 처리
    for batch_idx, batch in enumerate(batches):
        # ThreadPoolExecutor로 병렬 처리
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # ...
```

## 설치 및 환경 설정 상세 가이드

### 1. 필수 패키지 설치
```bash
pip install pandas numpy matplotlib seaborn scikit-learn nltk tqdm
pip install openpyxl python-docx olefile python-pptx PyPDF2
```

### 2. 추가 NLTK 리소스 다운로드
```python
import nltk
nltk.download('punkt', quiet=True)  # quiet=True 파라미터 추가로 다운로드 출력 최소화
```

### 3. 경로 설정
코드 내에서 `BASE_PATH` 변수를 분석할 문서가 있는 디렉토리로 설정합니다:
```python
# 사용자 환경에 맞게 수정 필요
BASE_PATH = Path(r"C:\Users\your_username\Desktop\IBK_Documents")
```

### 4. 환경 확인 기능 활용
코드에 포함된 `prepare_environment()` 함수를 사용하여 필요한 라이브러리가 올바르게 설치되었는지 확인할 수 있습니다:

```python
# 환경 준비 확인
if prepare_environment():
    print("모든 필수 라이브러리가 설치되어 있습니다.")
else:
    print("일부 라이브러리가 설치되지 않았습니다. 위 안내에 따라 설치해주세요.")
```

### 5. 추가 패키지 요구사항
코드 실행을 위해 다음 openpyxl 스타일 관련 모듈이 필요합니다:

```python
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.formatting.rule import CellIsRule
from openpyxl.utils import get_column_letter
```

### 6. 병렬 처리 최적화
대량의 파일을 처리하는 경우, 코드에서 사용되는 병렬 처리 설정을 조정할 수 있습니다:

```python
# 병렬 처리 스레드 수 조정 (CPU 코어 수에 따라 최적화)
max_workers = min(os.cpu_count() or 4, 8)  # 최대 8개 워커
```

### 7. 메모리 관리
대용량 파일 처리 시 메모리 사용량을 관리하기 위한 설정:

```python
# 대용량 파일 처리 크기 제한 설정
batch_size = 200  # 한 번에 처리할 파일 수
max_text_size = 100000  # 파일당 최대 처리 텍스트 크기
```

## 코드 실행 및 커스터마이징 가이드

### 1. 전체 분석 실행
노트북의 마지막 셀을 실행하면 모든 분석이 자동으로 진행됩니다:
```python
# 계층적 하이브리드 방식 적용 - 준법감시필 번호 및 유효기간 추출
try:
    # 라이브러리 임포트
    # 파일 분석 및 결과 처리
    # 보고서 생성 및 통계 출력
    # ...
except Exception as e:
    print(f"오류 발생: {e}")
    print(traceback.format_exc())
```

### 2. 키워드 및 패턴 커스터마이징
준법감시 관련 키워드와 패턴을 수정하려면 다음 부분을 변경합니다:
```python
# 키워드 설정
self.compliance_keywords = [
    '준법감시', '준법감시인', '준법감사', '심의필', '제호', 
    '승인', '결재', '법규', '컴플라이언스'
    # 여기에 추가 키워드 입력
]

self.validity_keywords = [
    '유효기간', '만료일', '유효', '만료', '기간', 
    '까지', '효력', '사용기한'
    # 여기에 추가 키워드 입력
]
```

### 3. 정규식 패턴 수정
파일명이나 문서 내용에서 유효기간을 추출하는 패턴을 수정하려면 다음 부분을 수정합니다:
```python
expiry_patterns = [
    r'유효기간\((\d{4})\.(\d{2})\.(\d{2})\)',  # 유효기간(2025.08.20)
    # 여기에 추가 패턴 입력
]
```

### 4. 오류 날짜 추가
특정 날짜가 오류로 판단되면 `ERROR_DATES` 목록에 추가합니다:
```python
ERROR_DATES = ['2024-11-30', '2020-12-31', '추가할_오류_날짜']
```

### 5. 디버깅 모드 설정
디버깅 정보를 상세하게 기록하려면 다음 설정을 활용합니다:
```python
# 디버깅 모드 활성화
DEBUG_MODE = True
DEBUG_FILE = "debug_log.txt"

def log_debug(message):
    if DEBUG_MODE:
        with open(DEBUG_FILE, "a", encoding="utf-8") as f:
            f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")
```

## 결과 해석 상세 가이드

### 1. Excel 보고서 구조
생성된 Excel 파일은 다음 열들을 포함합니다:

| 열 이름 | 설명 |
|--------|------|
| 파일명 | 분석된 문서 파일명 |
| 상태 | 유효기간 기준 문서 상태 (만료됨, 30일 이내 만료, 유효함, 상태 불명) |
| 만료일 | 추출된 유효기간 날짜 (YYYY-MM-DD 형식) |
| 남은 일수 | 현재 날짜 기준 만료일까지 남은 일수 |
| 파일명_준법감시필 | 파일명에서 추출한 준법감시필 번호 |
| 파일명_유효기간 | 파일명에서 추출한 유효기간 날짜 |
| 파일명_유효기간_상태 | 파일명 기준 유효기간 상태 |
| 파일명_남은일수 | 파일명 기준 남은 일수 |
| 파일내_준법감시필_번호 | 문서 내용에서 추출한 준법감시필 번호 |
| 파일내_유효기간_날짜 | 문서 내용에서 추출한 유효기간 날짜 |
| 파일내_유효기간_상태 | 문서 내용 기준 유효기간 상태 |
| 파일내_남은일수 | 문서 내용 기준 남은 일수 |
| 파일경로 | 파일이 위치한 경로 |

### 2. 분석 통계 해석
프로그램 실행이 완료되면 다음과 같은 통계 정보가 표시됩니다:

```
=== 분석 결과 통계 ===
- 만료됨: 267개
- 30일 이내 만료: 0개
- 유효함: 392개
- 상태 불명: 0개
- 총 파일 수: 659개

=== 정보 출처 통계 ===
- 파일명에서만 발견: 583개 (88.5%)
- 파일 내용에서만 발견: 0개 (0.0%)
- 파일명과 내용 모두에서 발견: 76개 (11.5%)
- 어디에서도 발견되지 않음: 0개 (0.0%)
```

이 통계는 분석된 문서의 전반적인 상태와 정보 출처를 보여줍니다.

## 문제 해결 및 디버깅

### 1. 디버그 로그 활용
코드에는 상세한 디버그 로그 기능이 포함되어 있습니다:
```python
DEBUG_MODE = True
DEBUG_FILE = "debug_log.txt"

def log_debug(message):
    if DEBUG_MODE:
        with open(DEBUG_FILE, "a", encoding="utf-8") as f:
            f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")
```

이 로그 파일을 확인하여 파일 처리 과정과, 추출 결과 등을 자세히 확인할 수 있습니다.

### 2. 특정 파일 분석 오류
특정 파일에서 오류가 발생하면 디버그 로그에서 해당 파일 처리 과정을 확인할 수 있습니다. 파일 인덱스 정보도 함께 저장되므로 문제가 발생한 파일을 쉽게 식별할 수 있습니다.

### 3. 메모리 사용량 최적화
대용량 파일이나 많은 수의 파일 처리 시 메모리 부족 오류가 발생할 수 있습니다. 이런 경우 `batch_size` 변수를 줄이거나, `max_normal_files` 값을 조정하여 처리할 파일 수를 제한할 수 있습니다.

이 도구를 통해 IBK 준법감시 문서의 유효기간을 효율적으로 관리하고, 만료되었거나 만료 예정인 문서를 사전에 식별하여 규제 준수 및 리스크 관리를 강화할 수 있습니다.

1. 필요한 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.util import ngrams
import hashlib
import concurrent.futures
import traceback
import json
import gc
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.formatting.rule import CellIsRule
from openpyxl.utils import get_column_letter
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# API 키 가져오기
api_key = os.getenv("api_key")
# 경고 메시지 설정
warnings.filterwarnings('ignore')

# NLTK 필요 패키지 다운로드
nltk.download('punkt', quiet=True)

# 기본 경로 설정
BASE_PATH = Path(r"C:\Users\markcloud\Desktop\오준호\IBK")

# 디버깅 모드 활성화
DEBUG_MODE = True
DEBUG_FILE = "debug_log.txt"

# 오류 날짜 목록 - 이 날짜들은 검사에서 제외됩니다
ERROR_DATES = ['2024-11-30', '2020-12-31']

# 디버깅 로그 함수
def log_debug(message):
    if DEBUG_MODE:
        with open(DEBUG_FILE, "a", encoding="utf-8") as f:
            f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")
            
# 디버그 로그 파일 초기화
with open(DEBUG_FILE, "w", encoding="utf-8") as f:
    f.write(f"=== 디버그 로그 시작: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ===\n")

#2. 파일 이름 분석 및 사전 필터링 함수

In [ ]:
def has_keywords_in_name(filename):
    """파일명에 준법감시 관련 키워드가 있는지 확인"""
    keywords = ['유효기간', '준법감시', '심의필', '만료일', '법무', '결재', '승인']
    return any(keyword in filename for keyword in keywords)

def extract_expiry_from_filename_improved(filename):
    """파일명에서 유효기간 정보를 추출 - 개선된 버전"""
    import re
    from datetime import datetime
    
    log_debug(f"파일명 분석 시작: {filename}")
    
    # IBK 파일명 패턴에 맞는 정규식
    ibk_patterns = [
        r'유효기간\(([0-9]{4})[\.\\-]([0-9]{2})[\.\\-]([0-9]{2})\)',  # 유효기간(2025.08.20)
        r'유효기간\(([0-9]{4})([0-9]{2})([0-9]{2})\)',  # 유효기간(20250820)
        r'유효기간\(([0-9]{4})[-/.]([0-9]{1,2})[-/.]([0-9]{1,2})\)',  # 유효기간(2025-08-20)
        r'제[0-9]{4}-[0-9]+호\([^)]+\)\s*유효기간\((\d{4})[./-](\d{1,2})[./-](\d{1,2})\)',  # 제2024-4806호(날짜) 유효기간(2025.08.20)
        r'유효기간[_\s]([0-9]{4})[./-]([0-9]{1,2})[./-]([0-9]{1,2})',  # 유효기간_2025.08.20
        r'([0-9]{4})[./-]([0-9]{1,2})[./-]([0-9]{1,2})[_\s]유효기간',  # 2025.08.20_유효기간
        r'만료일[_\s]([0-9]{4})[./-]([0-9]{1,2})[./-]([0-9]{1,2})'   # 만료일_2025.08.20
    ]
    
    # 패턴 검색
    for i, pattern in enumerate(ibk_patterns):
        match = re.search(pattern, filename)
        if match:
            try:
                # 패턴에 따라 그룹 인덱스 조정
                if '제' in pattern and len(match.groups()) >= 3:  # 제2024-4806호 형식
                    year, month, day = map(int, match.groups()[-3:])
                else:  # 일반 형식
                    year, month, day = map(int, match.groups())
                    
                if 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                    expiry_date = datetime(year, month, day)
                    log_debug(f"유효기간 추출 성공: {expiry_date.strftime('%Y-%m-%d')} (패턴 {i+1})")
                    
                    # 오류 날짜 필터링
                    date_str = expiry_date.strftime('%Y-%m-%d')
                    if date_str in ERROR_DATES:
                        log_debug(f"오류 날짜 감지: {date_str}")
                        continue
                        
                    return expiry_date
            except (ValueError, IndexError) as e:
                log_debug(f"패턴 {i+1} 매칭 오류: {str(e)}")
                continue
    
    # 확장 패턴 시도 (날짜 형식이 다른 경우)
    extended_patterns = [
        r'유효기간[:\s]*([12]\d{3})[년\.\-]?([01]?\d)[월\.\-]?([0-3]?\d)[일]?',  # 유효기간: 2025년6월3일
        r'([12]\d{3})년?[^0-9]+([01]?\d)월?[^0-9]+([0-3]?\d)일?[^0-9]*유효',  # 2025년 6월 3일 유효
        r'유효[^0-9]*([12]\d{3})년?[^0-9]+([01]?\d)월?[^0-9]+([0-3]?\d)일?'   # 유효 2025년 6월 3일
    ]
    
    for i, pattern in enumerate(extended_patterns):
        match = re.search(pattern, filename)
        if match:
            try:
                groups = match.groups()
                log_debug(f"확장 패턴 {i+1} 매치: {pattern} -> {groups}")
                
                year, month, day = map(int, groups)
                if 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                    expiry_date = datetime(year, month, day)
                    date_str = expiry_date.strftime('%Y-%m-%d')
                    
                    if date_str in ERROR_DATES:
                        log_debug(f"오류 날짜 감지: {date_str}")
                        continue
                        
                    log_debug(f"확장 패턴으로 유효기간 추출 성공: {date_str}")
                    return expiry_date
            except (ValueError, IndexError) as e:
                log_debug(f"확장 패턴 {i+1} 매칭 오류: {str(e)}")
                continue
    
    log_debug(f"파일명에서 유효기간 추출 실패: {filename}")
    return None

def extract_info_from_filename(filename):
    """파일명에서 준법감시 정보 추출 (준법감시필 번호 + 유효기간)"""
    # 유효기간 추출
    expiry_date = extract_expiry_from_filename_improved(filename)
    
    # 준법감시필 번호 추출
    compliance_number = extract_compliance_number(filename)
    
    # 유효기간 상태 계산
    if expiry_date:
        today = datetime.now().date()
        expiry_date_obj = expiry_date.date()
        days_to_expiry = (expiry_date_obj - today).days
        
        if days_to_expiry < 0:
            status = '만료됨'
        elif days_to_expiry <= 30:
            status = '30일 이내 만료'
        else:
            status = '유효함'
    else:
        days_to_expiry = None
        status = '상태 불명'
    
    return {
        'filename': filename,
        'expiry_date': expiry_date.strftime('%Y-%m-%d') if expiry_date else None,
        'compliance_number': compliance_number,
        'days_to_expiry': days_to_expiry,
        'status': status,
        'source': '파일명'
    }

def extract_compliance_number(filename):
    """파일명에서 준법감시필 번호 추출"""
    log_debug(f"준법감시필 번호 추출 시작: {filename}")
    
    compliance_patterns = [
        r'제(\d{4})-(\d+)호',  # 제2024-4806호
        r'준법감시[_\-\s]*(\d{4})[_\-\s]*(\d+)',  # 준법감시-2024-123
        r'심의필[_\-\s]*(\d{4})[_\-\s]*(\d+)',    # 심의필-2024-123
        r'준법[_\-\s]*(\d{4})[_\-\s]*(\d+)'       # 준법-2024-123
    ]
    
    for i, pattern in enumerate(compliance_patterns):
        match = re.search(pattern, filename)
        if match:
            try:
                year, number = match.groups()
                
                # 형식에 따라 준법감시필 번호 생성
                if pattern.startswith(r'제'):
                    compliance_number = f"제{year}-{number}호"
                elif pattern.startswith(r'준법감시'):
                    compliance_number = f"준법감시-{year}-{number}"
                elif pattern.startswith(r'심의필'):
                    compliance_number = f"심의필-{year}-{number}"
                else:
                    compliance_number = f"준법-{year}-{number}"
                
                log_debug(f"준법감시필 번호 추출 성공: {compliance_number} (패턴 {i+1})")
                return compliance_number
            except Exception as e:
                log_debug(f"준법감시필 번호 추출 오류: {str(e)}")
                continue
    
    log_debug(f"파일명에서 준법감시필 번호 추출 실패: {filename}")
    return None

#3. 문서 로드 및 전처리 함수

In [ ]:
def read_file_content(file_path):
    """다양한 파일 형식에서 텍스트 추출"""
    content = ""
    ext = file_path.suffix.lower()
    
    try:
        # 텍스트 파일
        if ext == '.txt':
            try:
                with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                    content = f.read(100000)  # 최대 10만자만 읽기
            except:
                try:
                    with open(file_path, 'r', encoding='cp949', errors='replace') as f:
                        content = f.read(100000)
                except:
                    pass
        
        # Word 문서
        elif ext == '.docx':
            try:
                import docx
                doc = docx.Document(file_path)
                
                # 모든 텍스트 추출 (단락, 테이블, 헤더/푸터)
                all_text = []
                
                # 1. 단락 추출
                for para in doc.paragraphs:
                    if para.text.strip():
                        all_text.append(para.text)
                
                # 2. 테이블 추출
                for table in doc.tables:
                    for row in table.rows:
                        row_text = ' | '.join([cell.text.strip() for cell in row.cells if cell.text.strip()])
                        if row_text:
                            all_text.append(row_text)
                
                # 3. 헤더/푸터 추출 (가능한 경우)
                try:
                    for section in doc.sections:
                        if section.header:
                            for p in section.header.paragraphs:
                                if p.text.strip():
                                    all_text.append(p.text)
                        if section.footer:
                            for p in section.footer.paragraphs:
                                if p.text.strip():
                                    all_text.append(p.text)
                except:
                    pass
                
                content = "\n".join(all_text)
                if len(all_text) > 0:
                    log_debug(f"DOCX 파일 {file_path.name}에서 {len(all_text)}개 텍스트 블록 추출됨")
            except Exception as e:
                log_debug(f"DOCX 처리 오류({file_path.name}): {str(e)}")
        
        # Excel 파일
        elif ext in ['.xlsx', '.xls']:
            try:
                xl = pd.ExcelFile(file_path, engine='openpyxl')
                sheet_names = xl.sheet_names[:5]  # 최대 5개 시트만 처리
                
                all_text = []
                for sheet_name in sheet_names:
                    try:
                        sheet_df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl', nrows=1000)
                        # 데이터프레임을 문자열로 변환할 때 좀 더 철저히 처리
                        sheet_df = sheet_df.fillna('')  # NaN 값을 빈 문자열로 변환
                        
                        # 모든 데이터를 문자열화
                        for col in sheet_df.columns:
                            sheet_df[col] = sheet_df[col].astype(str)
                            sheet_df[col] = sheet_df[col].str.strip()
                        
                        # 행별로 데이터 추출
                        for _, row in sheet_df.iterrows():
                            # 빈 값 제외하고 결합
                            row_values = [val for val in row.values if val and val.strip()]
                            if row_values:
                                all_text.append(' | '.join(row_values))
                    except Exception as sheet_e:
                        log_debug(f"Excel 시트 '{sheet_name}' 처리 오류: {str(sheet_e)}")
                        continue
                
                content = '\n'.join(all_text)
                if len(all_text) > 0:
                    log_debug(f"Excel 파일 {file_path.name}에서 {len(all_text)}개 행 추출됨")
            except Exception as e:
                log_debug(f"Excel 처리 오류({file_path.name}): {str(e)}")
                pass
        
        # PowerPoint 파일
        elif ext == '.pptx':
            try:
                from pptx import Presentation
                prs = Presentation(file_path)
                slide_texts = []
                
                # 슬라이드별 처리
                for i, slide in enumerate(prs.slides):
                    # 슬라이드 번호 추가
                    slide_texts.append(f"---슬라이드 {i+1}---")
                    
                    # 모든 도형에서 텍스트 추출
                    for shape in slide.shapes:
                        # 텍스트가 있는 도형
                        if hasattr(shape, "text") and shape.text.strip():
                            slide_texts.append(shape.text)
                        
                        # 테이블 처리
                        if hasattr(shape, "has_table") and shape.has_table:
                            for row in shape.table.rows:
                                row_text = ' | '.join([cell.text.strip() for cell in row.cells if cell.text.strip()])
                                if row_text:
                                    slide_texts.append(row_text)
                
                content = "\n".join(slide_texts)
                if len(slide_texts) > 0:
                    log_debug(f"PPTX 파일 {file_path.name}에서 {len(slide_texts)}개 텍스트 블록 추출됨")
            except Exception as e:
                log_debug(f"PPTX 처리 오류({file_path.name}): {str(e)}")
        
        # HWP 파일
        elif ext == '.hwp':
            try:
                import olefile
                if olefile.isOleFile(str(file_path)):
                    hwp_content = []
                    
                    with olefile.OleFile(str(file_path)) as ole:
                        # 스트림 목록 확인
                        streams = ole.listdir()
                        log_debug(f"HWP 파일 {file_path.name} 스트림: {streams}")
                        
                        # PrvText 스트림 처리 (미리보기 텍스트)
                        if ole.exists('PrvText'):
                            try:
                                prv_text = ole.openstream('PrvText')
                                prv_bytes = prv_text.read()
                                prv_text.close()
                                
                                # 여러 인코딩 시도
                                for encoding in ['utf-16-le', 'cp949', 'euc-kr']:
                                    try:
                                        decoded = prv_bytes.decode(encoding, errors='replace')
                                        if decoded.strip():
                                            hwp_content.append(decoded)
                                            log_debug(f"HWP 미리보기 추출 성공({encoding}): {len(decoded)}자")
                                            break
                                    except:
                                        continue
                            except Exception as e:
                                log_debug(f"HWP 미리보기 추출 오류: {str(e)}")
                        
                        # 문서 요약 정보 처리
                        if ole.exists('HwpSummaryInformation'):
                            try:
                                summary = ole.openstream('HwpSummaryInformation')
                                summary_bytes = summary.read()
                                summary.close()
                                
                                # 가능한 인코딩으로 시도
                                for encoding in ['utf-16-le', 'cp949', 'euc-kr']:
                                    try:
                                        # 인코딩 변환 후 유효한 문자만 필터링
                                        decoded = summary_bytes.decode(encoding, errors='replace')
                                        valid_text = ''.join(ch for ch in decoded if ch.isprintable())
                                        if valid_text.strip():
                                            hwp_content.append(valid_text)
                                            log_debug(f"HWP 요약정보 추출 성공({encoding}): {len(valid_text)}자")
                                            break
                                    except:
                                        continue
                            except Exception as e:
                                log_debug(f"HWP 요약정보 추출 오류: {str(e)}")
                    
                    # 결과 합치기
                    content = "\n".join(hwp_content)
                    if content:
                        log_debug(f"HWP 파일 {file_path.name}에서 {len(content)}자 추출됨")
                else:
                    log_debug(f"HWP 파일 {file_path.name}은 OLE 형식이 아님")
            except Exception as e:
                log_debug(f"HWP 처리 오류({file_path.name}): {str(e)}")
                
        # PDF 파일
        elif ext == '.pdf':
            try:
                import PyPDF2
                with open(file_path, 'rb') as file:
                    pdf_content = []
                    
                    try:
                        # 엄격하지 않은 모드로 PDF 읽기
                        reader = PyPDF2.PdfReader(file, strict=False)
                        
                        # 모든 페이지 처리
                        for page_num, page in enumerate(reader.pages):
                            try:
                                page_text = page.extract_text()
                                if page_text and page_text.strip():
                                    pdf_content.append(f"---페이지 {page_num+1}---")
                                    pdf_content.append(page_text)
                                    log_debug(f"PDF 페이지 {page_num+1} 추출 성공: {len(page_text)}자")
                            except Exception as page_e:
                                log_debug(f"PDF 페이지 {page_num+1} 처리 오류: {str(page_e)}")
                                continue
                        
                        content = "\n".join(pdf_content)
                        if len(pdf_content) > 0:
                            log_debug(f"PDF 파일 {file_path.name}에서 {len(pdf_content)}개 블록 추출됨")
                    except Exception as e:
                        log_debug(f"PDF 파일 구조 오류({file_path.name}): {str(e)}")
            except Exception as e:
                log_debug(f"PDF 처리 오류({file_path.name}): {str(e)}")
        
        # 내용이 추출되었는지 확인하고 준법감시 키워드 확인
        if content:
            keywords = ['준법감시', '심의필', '준법감사', '유효기간', '만료일']
            found_keywords = [kw for kw in keywords if kw in content]
            
            if found_keywords:
                log_debug(f"파일 {file_path.name}에서 키워드 발견: {', '.join(found_keywords)}")
            else:
                log_debug(f"파일 {file_path.name}에서 키워드 미발견 (내용 길이: {len(content)}자)")
        else:
            log_debug(f"파일 {file_path.name}에서 내용 추출 실패")

        return content
        
    except Exception as e:
        log_debug(f"파일 {file_path.name} 읽기 오류: {str(e)}")
        return ""

def process_hwp(file_path):
    """향상된 HWP 파일 처리"""
    content = ""
    
    # 방법 1: olefile 사용
    try:
        import olefile
        if olefile.isOleFile(str(file_path)):
            hwp_content = []
            
            try:
                with olefile.OleFile(str(file_path)) as ole:
                    # 모든 스트림 확인
                    streams = ole.listdir()
                    log_debug(f"HWP 파일 '{file_path.name}' 스트림: {streams}")
                    
                    # PrvText 스트림 (미리보기) 처리
                    if ole.exists('PrvText'):
                        try:
                            stream = ole.openstream('PrvText')
                            data = stream.read()
                            stream.close()
                            
                            # 여러 인코딩 시도
                            for encoding in ['utf-16-le', 'utf-8', 'cp949', 'euc-kr']:
                                try:
                                    text = data.decode(encoding, errors='replace')
                                    if text and len(text.strip()) > 10:
                                        hwp_content.append(f"[미리보기 텍스트 - {encoding}]")
                                        hwp_content.append(text)
                                        log_debug(f"HWP 미리보기 텍스트 성공({encoding}): {len(text)}자")
                                        break
                                except:
                                    continue
                        except Exception as e:
                            log_debug(f"HWP 미리보기 텍스트 추출 오류: {str(e)}")
                    
                    # HwpSummaryInformation 스트림 처리
                    if ole.exists('HwpSummaryInformation'):
                        try:
                            stream = ole.openstream('HwpSummaryInformation')
                            data = stream.read()
                            stream.close()
                            
                            # 여러 인코딩 시도
                            for encoding in ['utf-16-le', 'utf-8', 'cp949', 'euc-kr']:
                                try:
                                    text = data.decode(encoding, errors='replace')
                                    # 출력 가능한 문자만 필터링
                                    text = ''.join(c for c in text if c.isprintable() or c in '\n\r\t')
                                    if text and len(text.strip()) > 5:
                                        hwp_content.append(f"[문서 요약 정보 - {encoding}]")
                                        hwp_content.append(text)
                                        log_debug(f"HWP 요약 정보 성공({encoding}): {len(text)}자")
                                        break
                                except:
                                    continue
                        except Exception as e:
                            log_debug(f"HWP 요약 정보 추출 오류: {str(e)}")
                    
                    # BodyText 관련 스트림 검색 
                    body_streams = [s for s in streams if 'BodyText' in str(s) or 'Section' in str(s)]
                    for stream_name in body_streams:
                        try:
                            stream_path = '/'.join(stream_name) if isinstance(stream_name, tuple) else stream_name
                            stream = ole.openstream(stream_path)
                            data = stream.read()
                            stream.close()
                            
                            # 여러 인코딩 시도
                            for encoding in ['utf-16-le', 'utf-8', 'cp949', 'euc-kr']:
                                try:
                                    text = data.decode(encoding, errors='replace')
                                    # 출력 가능한 문자만 필터링
                                    text = ''.join(c for c in text if c.isprintable() or c in '\n\r\t')
                                    # 의미 있는 텍스트 필터링 (너무 짧거나 의미 없는 문자열 제외)
                                    if text and len(text.strip()) > 20 and any(c.isalpha() for c in text):
                                        hwp_content.append(f"[본문 스트림 - {stream_path}]")
                                        hwp_content.append(text)
                                        log_debug(f"HWP 본문 스트림 성공({encoding}): {len(text)}자")
                                        break
                                except:
                                    continue
                        except Exception as e:
                            log_debug(f"HWP 본문 스트림 처리 오류({stream_name}): {str(e)}")
                
                # 결과 합치기
                if hwp_content:
                    content = "\n".join(hwp_content)
                    log_debug(f"HWP 파일 '{file_path.name}'에서 올리파일로 {len(hwp_content)}개 블록 추출 성공")
            except Exception as e:
                log_debug(f"HWP 올리파일 처리 오류: {str(e)}")
        else:
            log_debug(f"HWP 파일 '{file_path.name}'은 OLE 형식이 아님")
    except ImportError:
        log_debug("olefile 라이브러리 없음, 설치 필요: pip install olefile")
    except Exception as e:
        log_debug(f"HWP 올리파일 방식 오류: {str(e)}")
    
    # 방법 2: hwp-to-txt 유틸리티 사용 (필요 시)
    if not content or len(content) < 100:
        try:
            # 외부 명령어 시도 (hwp5txt 필요)
            import subprocess
            result = subprocess.run(['hwp5txt', str(file_path)], capture_output=True, text=True, encoding='utf-8')
            if result.stdout and len(result.stdout) > len(content):
                content = result.stdout
                log_debug(f"HWP 파일 '{file_path.name}'에서 hwp5txt로 변환 성공: {len(content)}자")
        except Exception as e:
            log_debug(f"hwp5txt 변환 시도 실패: {str(e)}")
    
    # 방법 3: 관련된 TXT 파일 찾기
    if not content or len(content) < 100:
        try:
            # 같은 이름의 TXT 파일이 있는지 확인
            txt_path = file_path.with_suffix('.txt')
            if txt_path.exists():
                with open(txt_path, 'r', encoding='utf-8', errors='replace') as f:
                    txt_content = f.read()
                    if txt_content and len(txt_content) > len(content):
                        content = txt_content
                        log_debug(f"HWP 관련 TXT 파일 '{txt_path.name}'에서 읽기 성공: {len(content)}자")
        except Exception as e:
            log_debug(f"관련 TXT 파일 읽기 실패: {str(e)}")
    
    return content
def process_txt(file_path):
    """향상된 TXT 파일 처리"""
    content = ""
    
    # 여러 인코딩으로 시도
    encodings = ['utf-8', 'cp949', 'euc-kr', 'latin1']
    success = False
    
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding, errors='replace') as f:
                text = f.read()
                
                if text and text.strip():
                    # 기본적인 텍스트 정규화
                    content = text
                    log_debug(f"TXT 파일 '{file_path.name}'을 {encoding} 인코딩으로 성공적으로 읽음: {len(content)}자")
                    success = True
                    break
        except UnicodeDecodeError:
            continue
        except Exception as e:
            log_debug(f"TXT 파일 '{file_path.name}' {encoding} 인코딩 시도 중 오류: {str(e)}")
    
    if not success:
        try:
            # 바이너리 모드로 읽어서 인코딩 추측
            with open(file_path, 'rb') as f:
                raw_data = f.read()
                
                # 한글 인코딩 추측을 위한 간단한 휴리스틱
                if raw_data.startswith(b'\xff\xfe') or raw_data.startswith(b'\xfe\xff'):
                    # UTF-16 인코딩
                    content = raw_data.decode('utf-16', errors='replace')
                    log_debug(f"TXT 파일 '{file_path.name}'을 UTF-16 인코딩으로 읽음: {len(content)}자")
                else:
                    # 마지막 시도: 바이너리 데이터에서 텍스트 추출
                    content = raw_data.decode('utf-8', errors='replace')
                    log_debug(f"TXT 파일 '{file_path.name}'을 바이너리에서 강제 변환: {len(content)}자")
        except Exception as e:
            log_debug(f"TXT 파일 '{file_path.name}' 최종 시도 오류: {str(e)}")
    
    return content

def process_docx(file_path):
    """향상된 DOCX 파일 처리"""
    content = ""
    try:
        # 라이브러리 확인 및 설치
        try:
            import docx
        except ImportError:
            print("python-docx 라이브러리가 설치되지 않았습니다. 설치 중...")
            import subprocess
            subprocess.check_call(["pip", "install", "python-docx"])
            import docx
        
        # 문서 열기
        doc = docx.Document(str(file_path))
        
        # 1. 모든 단락 추출 
        paragraphs_text = []
        for para in doc.paragraphs:
            if para.text.strip():
                # 텍스트 정규화 (줄바꿈 및 공백 처리)
                clean_text = re.sub(r'\s+', ' ', para.text).strip()
                if clean_text:
                    paragraphs_text.append(clean_text)
        
        # 2. 모든 표 추출
        tables_text = []
        for table in doc.tables:
            for row in table.rows:
                row_text = []
                for cell in row.cells:
                    if cell.text.strip():
                        row_text.append(cell.text.strip())
                if row_text:
                    tables_text.append(' | '.join(row_text))
        
        # 3. 헤더 및 푸터 추출 시도
        header_footer_text = []
        try:
            for section in doc.sections:
                # 헤더 추출
                if section.header:
                    for p in section.header.paragraphs:
                        if p.text.strip():
                            header_footer_text.append("[헤더] " + p.text.strip())
                
                # 푸터 추출
                if section.footer:
                    for p in section.footer.paragraphs:
                        if p.text.strip():
                            header_footer_text.append("[푸터] " + p.text.strip())
        except Exception as e:
            log_debug(f"DOCX 헤더/푸터 추출 오류: {str(e)}")
        
        # 4. 수정 이력 및 코멘트 추출 시도
        comments_text = []
        try:
            if hasattr(doc, 'comments'):
                for comment in doc.comments:
                    if comment.text.strip():
                        comments_text.append("[코멘트] " + comment.text.strip())
        except Exception as e:
            log_debug(f"DOCX 코멘트 추출 오류: {str(e)}")
        
        # 모든 텍스트 결합
        all_texts = paragraphs_text + tables_text + header_footer_text + comments_text
        
        if all_texts:
            content = "\n".join(all_texts)
            log_debug(f"DOCX 파일 '{file_path.name}'에서 {len(all_texts)}개 텍스트 블록 추출 성공")
        else:
            log_debug(f"DOCX 파일 '{file_path.name}'에서 추출된 텍스트 없음")
    
    except Exception as e:
        log_debug(f"DOCX 파일 '{file_path.name}' 처리 중 오류: {str(e)}")
    
    return content

def collect_target_files(base_path):
    """타겟 파일 수집 - 병렬 처리 최적화"""
    print("분석할 파일 찾는 중...")
    file_extensions = ['.docx', '.xlsx', '.pptx', '.hwp', '.txt', '.pdf']
    
    target_files = []
    
    # 디렉토리 목록 생성
    all_dirs = []
    try:
        for d in Path(base_path).iterdir():
            if d.is_dir():
                all_dirs.append(d)
    except Exception as e:
        print(f"디렉토리 목록 생성 중 오류: {e}")
    
    if not all_dirs:  # 하위 디렉토리가 없으면 현재 디렉토리 검색
        all_dirs = [Path(base_path)]
    
    print(f"총 {len(all_dirs)}개 디렉토리 검색 예정")
    
    # 병렬 파일 검색 - 간소화 버전
    def search_files_in_directory(directory):
        found_files = []
        for ext in file_extensions:
            try:
                for file_path in Path(directory).glob(f'**/*{ext}'):
                    # 키워드 필터링 조건 제거 - 모든 파일 포함
                    found_files.append(file_path)
            except Exception as e:
                print(f"디렉토리 {directory} 검색 중 오류: {e}")
        return found_files
    
    # 파일 검색 (병렬 처리)
    target_files = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        search_results = list(executor.map(search_files_in_directory, all_dirs))
        for result in search_results:
            target_files.extend(result)
    
    print(f"총 {len(target_files)}개 파일을 찾았습니다.")
    return target_files

In [ ]:
def extract_expiry_from_filename(filename):
    """파일명에서 유효기간 정보를 추출 - 개선된 버전"""
    import re
    from datetime import datetime
    
    # 디버깅 로그 함수가 없으면 간단히 구현
    try:
        log_debug(f"파일명 분석 시작: {filename}")
    except:
        def log_debug(msg): pass
    
    # IBK 파일명 패턴에 맞는 정규식
    ibk_patterns = [
        r'유효기간\(([0-9]{4})[\\.\\-]([0-9]{2})[\\.\\-]([0-9]{2})\)',  # 유효기간(2025.08.20)
        r'유효기간\(([0-9]{4})([0-9]{2})([0-9]{2})\)',  # 유효기간(20250820)
        r'유효기간\(([0-9]{4})[-/.]([0-9]{1,2})[-/.]([0-9]{1,2})\)',  # 유효기간(2025-08-20)
        r'제[0-9]{4}-[0-9]+호\([^)]+\)\s*유효기간\((\d{4})[./-](\d{1,2})[./-](\d{1,2})\)',  # 제2024-4806호(날짜) 유효기간(2025.08.20)
        r'유효기간[_\s]([0-9]{4})[./-]([0-9]{1,2})[./-]([0-9]{1,2})',  # 유효기간_2025.08.20
        r'([0-9]{4})[./-]([0-9]{1,2})[./-]([0-9]{1,2})[_\s]유효기간',  # 2025.08.20_유효기간
        r'만료일[_\s]([0-9]{4})[./-]([0-9]{1,2})[./-]([0-9]{1,2})'   # 만료일_2025.08.20
    ]
    
    # 오류 날짜 목록 - 전역변수가 없으면 기본값 설정
    try:
        ERROR_DATES
    except NameError:
        ERROR_DATES = ['2024-11-30', '2020-12-31']
    
    # 패턴 검색
    for i, pattern in enumerate(ibk_patterns):
        match = re.search(pattern, filename)
        if match:
            try:
                # 패턴에 따라 그룹 인덱스 조정
                if '제' in pattern and len(match.groups()) >= 3:  # 제2024-4806호 형식
                    year, month, day = map(int, match.groups()[-3:])
                else:  # 일반 형식
                    year, month, day = map(int, match.groups())
                    
                if 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                    expiry_date = datetime(year, month, day)
                    try:
                        log_debug(f"유효기간 추출 성공: {expiry_date.strftime('%Y-%m-%d')} (패턴 {i+1})")
                    except:
                        pass
                    
                    # 오류 날짜 필터링
                    date_str = expiry_date.strftime('%Y-%m-%d')
                    if date_str in ERROR_DATES:
                        try:
                            log_debug(f"오류 날짜 감지: {date_str}")
                        except:
                            pass
                        continue
                        
                    return expiry_date
            except (ValueError, IndexError) as e:
                try:
                    log_debug(f"패턴 {i+1} 매칭 오류: {str(e)}")
                except:
                    pass
                continue
    
    # 확장 패턴 시도 (날짜 형식이 다른 경우)
    extended_patterns = [
        r'유효기간[:\s]*([12]\d{3})[년\.\-]?([01]?\d)[월\.\-]?([0-3]?\d)[일]?',  # 유효기간: 2025년6월3일
        r'([12]\d{3})년?[^0-9]+([01]?\d)월?[^0-9]+([0-3]?\d)일?[^0-9]*유효',  # 2025년 6월 3일 유효
        r'유효[^0-9]*([12]\d{3})년?[^0-9]+([01]?\d)월?[^0-9]+([0-3]?\d)일?'   # 유효 2025년 6월 3일
    ]
    
    for i, pattern in enumerate(extended_patterns):
        match = re.search(pattern, filename)
        if match:
            try:
                groups = match.groups()
                try:
                    log_debug(f"확장 패턴 {i+1} 매치: {pattern} -> {groups}")
                except:
                    pass
                
                year, month, day = map(int, groups)
                if 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                    expiry_date = datetime(year, month, day)
                    date_str = expiry_date.strftime('%Y-%m-%d')
                    
                    if date_str in ERROR_DATES:
                        try:
                            log_debug(f"오류 날짜 감지: {date_str}")
                        except:
                            pass
                        continue
                        
                    try:
                        log_debug(f"확장 패턴으로 유효기간 추출 성공: {date_str}")
                    except:
                        pass
                    return expiry_date
            except (ValueError, IndexError) as e:
                try:
                    log_debug(f"확장 패턴 {i+1} 매칭 오류: {str(e)}")
                except:
                    pass
                continue
    
    try:
        log_debug(f"파일명에서 유효기간 추출 실패: {filename}")
    except:
        pass
    return None

def has_keywords_in_name(filename):
    """파일명에 준법감시 관련 키워드가 있는지 확인"""
    keywords = ['유효기간', '준법감시', '심의필', '만료일', '법무', '결재', '승인']
    return any(keyword in filename for keyword in keywords)

def load_documents(directory_path, file_extensions=['.txt', '.docx', '.hwp', '.xlsx', '.xls', '.pptx', '.ppt', '.csv', '.pdf']):
    """다양한 파일 형식에서 텍스트 추출 (PDF 제외)"""
    documents = []
    filenames = []
    file_contents = {}
    expiry_dates = {}  # 파일명에서 추출한 유효기간 정보 저장
    
    # 우선순위 파일과 일반 파일 분리
    priority_files = []
    normal_files = []
    
    print("파일 목록 수집 중...")
    # 모든 파일 탐색 및 분류
    for file_path in Path(directory_path).glob('**/*'):
        if not file_path.is_file():
            continue
            
        # 파일 확장자 확인
        ext = file_path.suffix.lower()
        if ext not in file_extensions:
            continue
                    
        # 임시 파일 건너뛰기
        if file_path.name.startswith('~$') or file_path.name.startswith('.~'):
            continue
        
        # 파일명에서 키워드 확인 및 우선순위 부여
        if has_keywords_in_name(file_path.name):
            priority_files.append(file_path)
            
            # 파일명에서 유효기간 정보 추출
            expiry_date = extract_expiry_from_filename(file_path.name)
            if expiry_date:
                expiry_dates[file_path.name] = expiry_date
        else:
            normal_files.append(file_path)
    
    print(f"우선 처리할 파일: {len(priority_files)}개")
    print(f"일반 처리할 파일: {len(normal_files)}개")
    print(f"파일명에서 유효기간 정보 추출: {len(expiry_dates)}개")
    
    # 처리할 파일 목록 준비 (우선순위 파일 + 일부 일반 파일)
    max_normal_files = min(5000, len(normal_files))  # 최대 5000개 일반 파일만 처리
    all_files = priority_files + normal_files[:max_normal_files]
    
    # 진행 상황 표시
    for file_path in tqdm(all_files, desc="파일 처리 중"):
        try:
            content = ""
            
            # 1. 텍스트 파일 처리
            if file_path.suffix.lower() == '.txt':
                with open(file_path, 'r', encoding='utf-8', errors='replace') as file:
                    content = file.read()
            
            # 2. CSV 파일 처리
            elif file_path.suffix.lower() == '.csv':
                try:
                    # 다양한 옵션으로 시도
                    for encoding in ['utf-8', 'cp949']:
                        for sep in [',', '\t', ';']:
                            try:
                                df = pd.read_csv(file_path, encoding=encoding, sep=sep)
                                if not df.empty:
                                    text_cols = [col for col in df.columns if df[col].dtype == 'object']
                                    if text_cols:
                                        content = '\n'.join(df[text_cols].fillna('').astype(str).apply(' '.join, axis=1).tolist())
                                    else:
                                        content = '\n'.join(df.fillna('').astype(str).apply(' '.join, axis=1).tolist())
                                    break
                            except:
                                continue
                        if content:
                            break
                except Exception as e:
                    print(f"CSV 파일 처리 오류: {file_path.name} - {e}")
            
            # 3. Word 파일 처리
            elif file_path.suffix.lower() == '.docx':
                try:
                    import docx
                    doc = docx.Document(file_path)
                    content = "\n".join([paragraph.text for paragraph in doc.paragraphs if paragraph.text])
                except ImportError:
                    print("python-docx 라이브러리가 필요합니다.")
                    # 주피터 노트북에서 라이브러리 설치를 시도
                    import sys
                    !{sys.executable} -m pip install python-docx
                    # 설치 후 다시 시도
                    try:
                        import docx
                        doc = docx.Document(file_path)
                        content = "\n".join([paragraph.text for paragraph in doc.paragraphs if paragraph.text])
                    except ImportError:
                        print("python-docx 설치 실패. 수동으로 설치해 주세요.")
                        content = ""
            
            # 4. HWP 파일 처리 부분 수정
            elif ext == '.hwp':
                try:
                    import olefile
                    if olefile.isOleFile(str(file_path)):
                        print(f"HWP 파일 처리 시작: {file_path.name}")
                        with olefile.OleFile(str(file_path)) as ole:
                            # 스트림 목록 확인 (디버깅용)
                            streams = ole.listdir()
                            print(f"HWP 스트림: {streams}")
                            
                            # 여러 가능한 스트림 이름 시도
                            hwp_streams = ['PrvText', 'HwpSummaryInformation', 'DocInfo', 'BodyText']
                            content_parts = []
                            
                            for stream_name in hwp_streams:
                                if ole.exists(stream_name):
                                    try:
                                        stream = ole.openstream(stream_name)
                                        stream_data = stream.read()
                                        
                                        # 여러 인코딩 시도
                                        for encoding in ['utf-16-le', 'cp949', 'euc-kr']:
                                            try:
                                                decoded = stream_data.decode(encoding, errors='replace')
                                                if not decoded.isspace() and decoded.strip():
                                                    content_parts.append(decoded)
                                                    print(f"HWP 스트림 '{stream_name}' 읽기 성공 ({encoding})")
                                                break
                                            except:
                                                continue
                                        
                                        stream.close()
                                    except Exception as e:
                                        print(f"HWP 스트림 '{stream_name}' 읽기 오류: {str(e)}")
                            
                            content = "\n".join(content_parts)
                            
                            # 내용이 적거나 없으면 pyhwp 등 다른 라이브러리 사용 고려
                            if len(content) < 100:
                                print(f"HWP 파일 '{file_path.name}'에서 내용 추출 제한적임 ({len(content)} 자)")
                except Exception as e:
                    print(f"HWP 처리 오류: {str(e)}")
                    content = ""

            # 5. Excel 파일 처리 부분 수정
            elif file_path.suffix.lower() in ['.xlsx', '.xls']:
                try:
                    import openpyxl
                    # ExcelFile 객체 사용
                    xl = pd.ExcelFile(file_path, engine='openpyxl')
                    sheet_names = xl.sheet_names
                    
                    all_text = []
                    for sheet_name in sheet_names:
                        sheet_df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
                        text_cols = [col for col in sheet_df.columns if sheet_df[col].dtype == 'object']
                        if text_cols:
                            sheet_text = '\n'.join(sheet_df[text_cols].fillna('').astype(str).apply(' '.join, axis=1).tolist())
                            all_text.append(sheet_text)
                    
                    content = '\n'.join(all_text)
                except Exception as xl_e:
                    print(f"Excel 파일 처리 오류: {file_path.name} - {xl_e}")
                    content = f"[Excel 오류: {file_path.name}]"
            
            # 6. PowerPoint 파일 처리
            elif file_path.suffix.lower() in ['.pptx', '.ppt']:
                try:
                    import pptx
                    presentation = pptx.Presentation(file_path)
                    text_runs = []
                    
                    # 슬라이드 별 텍스트 추출
                    for slide in presentation.slides:
                        for shape in slide.shapes:
                            if hasattr(shape, "text") and shape.text:
                                text_runs.append(shape.text)
                    
                    content = "\n".join(text_runs)
                except ImportError:
                    print("PowerPoint 파일 처리를 위해 python-pptx 라이브러리가 필요합니다.")
                except Exception as ppt_e:
                    print(f"PowerPoint 파일 처리 오류: {file_path.name} - {ppt_e}")
            
            # 내용이 있는 경우만 추가
            if content:
                documents.append(content)
                filenames.append(file_path.name)
                file_contents[file_path.name] = content
                
        except Exception as e:
            print(f"파일 {file_path.name} 처리 중 오류: {e}")
    
    print(f"총 {len(documents)}개 파일에서 텍스트 추출 완료")
    return documents, filenames, file_contents, expiry_dates

#4. IBK 준법감시 패턴 정의

In [ ]:
class EnhancedComplianceExtractor:
    def __init__(self, api_key=None, use_ngram=True, use_ai=False):
        self.api_key = api_key
        self.use_ngram = use_ngram
        self.use_ai = use_ai
        self.chunk_size = 1000  # 최적 청크 크기 (GPT API 토큰 한도 고려)
        self.chunk_overlap = 200  # 맥락 유지를 위한 중첩 크기
        
        # 키워드 및 패턴 정의
        self.compliance_keywords = [
            '준법감시', '준법감시인', '준법감사', '심의필', '제호', 
            '승인', '결재', '법규', '컴플라이언스'
        ]
        
        self.validity_keywords = [
            '유효기간', '만료일', '유효', '만료', '기간', 
            '까지', '효력', '사용기한'
        ]
        
    def create_semantic_chunks(self, text):
        """의미론적 청크 생성 - 문장/단락 단위 분할"""
        if not text or len(text) < self.chunk_size:
            return [text] if text else []
        
        # 단락 기반 분할 (더 의미있는 청크)
        paragraphs = re.split(r'\n\s*\n', text)
        
        chunks = []
        current_chunk = ""
        
        for para in paragraphs:
            # 단일 단락이 청크 크기보다 크면 문장으로 분할
            if len(para) > self.chunk_size:
                sentences = re.split(r'(?<=[.!?])\s+', para)
                for sentence in sentences:
                    if len(current_chunk) + len(sentence) <= self.chunk_size:
                        current_chunk += sentence + " "
                    else:
                        chunks.append(current_chunk.strip())
                        current_chunk = sentence + " "
            # 청크 크기 이내면 단락 단위로 추가
            elif len(current_chunk) + len(para) <= self.chunk_size:
                current_chunk += para + "\n\n"
            else:
                chunks.append(current_chunk.strip())
                current_chunk = para + "\n\n"
        
        # 마지막 청크 추가
        if current_chunk.strip():
            chunks.append(current_chunk.strip())
        
        # 청크 중첩 처리로 맥락 유지
        overlapped_chunks = []
        for i in range(len(chunks)):
            if i < len(chunks) - 1:
                # 현재 청크와 다음 청크의 일부를 중첩
                overlap_size = min(self.chunk_overlap, len(chunks[i]), len(chunks[i+1]))
                
                if i == 0:
                    overlapped_chunks.append(chunks[i])
                
                # 중첩 청크: 현재 청크의 끝 + 다음 청크의 시작
                current_end = chunks[i][-overlap_size:] if overlap_size > 0 else ""
                next_start = chunks[i+1][:overlap_size] if overlap_size > 0 else ""
                
                overlapped_chunks.append(current_end + next_start)
            
            if i == len(chunks) - 1:
                overlapped_chunks.append(chunks[i])
        
        return overlapped_chunks if overlapped_chunks else chunks
        
    def score_chunks_by_relevance(self, chunks):
        """준법감시 관련성 기준으로 청크 점수화 및 필터링"""
        scored_chunks = []
        
        for i, chunk in enumerate(chunks):
            # 키워드 기반 점수
            keyword_score = 0
            for keyword in self.compliance_keywords:
                keyword_score += chunk.lower().count(keyword.lower()) * 5
            
            for keyword in self.validity_keywords:
                keyword_score += chunk.lower().count(keyword.lower()) * 5
            
            # 정규식 패턴 기반 점수
            pattern_score = 0
            compliance_patterns = [
                r'제\d{4}-\d+호',  # 제2024-4806호
                r'준법감시[_\-\s]*\d{4}[_\-\s]*\d+',  # 준법감시-2024-123
                r'심의필[_\-\s]*\d{4}[_\-\s]*\d+',    # 심의필-2024-123
            ]
            
            date_patterns = [
                r'유효기간[^0-9]*\d{4}[-/.년\s]\d{1,2}[-/.월\s]\d{1,2}일?',
                r'만료일[^0-9]*\d{4}[-/.년\s]\d{1,2}[-/.월\s]\d{1,2}일?'
            ]
            
            for pattern in compliance_patterns + date_patterns:
                if re.search(pattern, chunk):
                    pattern_score += 20
            
            # 종합 점수 및 저장
            total_score = keyword_score + pattern_score
            if total_score > 0:  # 관련성 있는 청크만 저장
                scored_chunks.append((i, chunk, total_score))
        
        # 점수 기준 정렬 및 상위 청크 선택
        scored_chunks.sort(key=lambda x: x[2], reverse=True)
        
        # 상위 청크 반환 (최대 5개, 또는 점수 30 이상인 모든 청크)
        top_chunks = [(idx, chunk) for idx, chunk, score in scored_chunks if score >= 30]
        if len(top_chunks) > 5:
            top_chunks = top_chunks[:5]
        
        return top_chunks
    
    def call_gpt_api(self, chunk):
        """GPT API 호출하여 준법감시 정보 추출"""
        if not self.use_ai or not self.api_key:
            return None
        
        # API 호출 코드 (실제 사용 시 활성화)
        log_debug("GPT API 호출 (가정)")
        return {
            "compliance_number": None, 
            "expiry_date": None,
            "confidence": 0.0,
            "context": None
        }
    
    def extract_ngram_features(self, text, n=3):
        """N-gram 특성 추출"""
        from nltk.util import ngrams
        from nltk.tokenize import word_tokenize
        
        # 토큰화
        tokens = word_tokenize(text.lower())
        
        # n-gram 생성
        n_grams = list(ngrams(tokens, n))
        return ['_'.join(gram) for gram in n_grams]
    
    def extract_with_regex(self, text):
        """정규식으로 준법감시 정보 추출"""
        log_debug("정규식으로 준법감시 정보 추출 시작")
        result = {"compliance_number": None, "expiry_date": None}
        
        # 준법감시필 번호 추출
        compliance_patterns = [
            r'제(\d{4})-(\d+)호',  # 제2024-4806호
            r'준법감시[_\-\s]*(\d{4})[_\-\s]*(\d+)',  # 준법감시-2024-123
            r'심의필[_\-\s]*(\d{4})[_\-\s]*(\d+)',    # 심의필-2024-123
            r'준법[_\-\s]*(\d{4})[_\-\s]*(\d+)'       # 준법-2024-123
        ]
        
        for pattern in compliance_patterns:
            match = re.search(pattern, text)
            if match:
                try:
                    year, number = match.groups()
                    
                    # 형식에 따라 준법감시필 번호 생성
                    if pattern.startswith(r'제'):
                        result["compliance_number"] = f"제{year}-{number}호"
                    elif pattern.startswith(r'준법감시'):
                        result["compliance_number"] = f"준법감시-{year}-{number}"
                    elif pattern.startswith(r'심의필'):
                        result["compliance_number"] = f"심의필-{year}-{number}"
                    else:
                        result["compliance_number"] = f"준법-{year}-{number}"
                    
                    log_debug(f"정규식으로 준법감시필 번호 추출: {result['compliance_number']}")
                    break
                except:
                    continue
        
        # 유효기간 추출
        expiry_patterns = [
            r'유효기간[^0-9]*(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?',  # 유효기간: 2025.08.20
            r'만료일[^0-9]*(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?',   # 만료일: 2025.08.20
            r'(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?[^0-9]*까지',  # 2025.08.20까지
            r'(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?[^0-9]*만료',  # 2025.08.20 만료
            r'유효[^0-9]*(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?'   # 유효: 2025.08.20
        ]
        
        for pattern in expiry_patterns:
            match = re.search(pattern, text)
            if match:
                try:
                    year, month, day = map(int, match.groups())
                    if 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                        date_str = f"{year}-{month:02d}-{day:02d}"
                        
                        # 오류 날짜 필터링
                        if date_str in ERROR_DATES:
                            log_debug(f"오류 날짜 감지: {date_str}")
                            continue
                        
                        result["expiry_date"] = date_str
                        log_debug(f"정규식으로 유효기간 추출: {date_str}")
                        break
                except:
                    continue
                    
        return result
    
    def combine_results(self, regex_result, gpt_result, ngram_features):
        """여러 추출 결과 통합 - 가중치 기반"""
        # 초기화
        final_result = {
            "compliance_number": None,
            "expiry_date": None,
            "confidence": 0.0,
            "context": None,
            "ngram_result": {},
            "gpt_result": {}
        }
        
        # 정규식 결과 처리 (가중치: 0.6)
        if regex_result:
            final_result["compliance_number"] = regex_result.get("compliance_number")
            final_result["expiry_date"] = regex_result.get("expiry_date")
            final_result["confidence"] = 0.6
            final_result["ngram_result"] = regex_result
        
        # GPT 결과 처리 (가중치: 0.8) - GPT API가 활성화된 경우
        if gpt_result:
            gpt_confidence = gpt_result.get("confidence", 0.0) * 0.8
            
            # 신뢰도가 높은 경우에만 기존 결과 대체
            if gpt_confidence > final_result["confidence"]:
                final_result["compliance_number"] = gpt_result.get("compliance_number")
                final_result["expiry_date"] = gpt_result.get("expiry_date")
                final_result["confidence"] = gpt_confidence
                final_result["context"] = gpt_result.get("context")
            
            # 일치하는 경우 신뢰도 상승
            elif (final_result["compliance_number"] == gpt_result.get("compliance_number") or
                  final_result["expiry_date"] == gpt_result.get("expiry_date")):
                final_result["confidence"] += 0.2
                
            final_result["gpt_result"] = gpt_result
        
        # N-gram 결과 활용 (신뢰도 미세 조정)
        if ngram_features and (final_result["compliance_number"] or final_result["expiry_date"]):
            final_result["confidence"] = min(1.0, final_result["confidence"] + 0.1)
        
        return final_result
    
    def process_document(self, content):
        """문서 내용에서 준법감시필 번호와 유효기간 추출"""
        log_debug("문서 내용 분석 시작")
        
        if not content or not isinstance(content, str):
            return {"compliance_number": None, "expiry_date": None, "confidence": 0.0}
        
        # 1. 텍스트 청크 분할
        chunks = self.create_semantic_chunks(content)
        
        # 2. 관련성 높은 청크 선별
        relevant_chunks = self.score_chunks_by_relevance(chunks)
        
        # 청크가 없으면 전체 텍스트에 대해 간단 분석
        if not relevant_chunks and content:
            mini_chunk = content[:500]
            regex_result = self.extract_with_regex(mini_chunk)
            ai_result = self.call_gpt_api(mini_chunk) if self.use_ai else None
            ngram_features = self.extract_ngram_features(mini_chunk) if self.use_ngram else None
            return self.combine_results(regex_result, ai_result, ngram_features)
        
        # 청크별 결과 저장
        chunk_results = []
        
        # 3. 각 청크별 분석
        for idx, chunk in relevant_chunks:
            regex_result = self.extract_with_regex(chunk)
            ai_result = self.call_gpt_api(chunk) if self.use_ai else None
            ngram_features = self.extract_ngram_features(chunk) if self.use_ngram else None
            
            combined = self.combine_results(regex_result, ai_result, ngram_features)
            combined["chunk_index"] = idx
            chunk_results.append(combined)
        
        # 결과가 없으면 빈 결과 반환
        if not chunk_results:
            return {"compliance_number": None, "expiry_date": None, "confidence": 0.0}
        
        # 4. 최종 결과 선택 (신뢰도 기준)
        chunk_results.sort(key=lambda x: x.get("confidence", 0.0), reverse=True)
        return chunk_results[0]

5. IBK 준법감시 정보 및 유효기간 추출 클래스

In [ ]:
def analyze_file(file_path, target_files=None):
    """파일 정보를 분석하여 결과 딕셔너리 반환 - 강화된 버전"""
    try:
        filename = file_path.name
        
        # target_files가 제공된 경우에만 인덱스 계산, 아니면 -1 사용
        if target_files is not None:
            file_index = target_files.index(file_path) if file_path in target_files else -1
            total_files = len(target_files)
        else:
            file_index = -1
            total_files = 0
        
        # 파일 순번 로깅
        log_debug(f"{'='*50}")
        log_debug(f"파일 분석 시작 [{file_index+1}/{total_files}]: {filename}")
        
        # 파일명에서 정보 추출
        filename_info = extract_info_from_filename(filename)
        
        # 파일 내용 확인 변수 초기화
        content_result = {
            "compliance_number": None,
            "expiry_date": None,
            "confidence": 0.0,
            "context": None
        }
        
        # 파일 확장자 확인
        ext = file_path.suffix.lower()
        
        # 파일 크기 확인
        try:
            file_size = file_path.stat().st_size / (1024 * 1024)  # MB 단위
            is_large_file = file_size > 10  # 10MB 초과
            
            if is_large_file:
                log_debug(f"대용량 파일 감지: {file_size:.2f}MB - 내용 분석 제한")
                content = read_file_content(file_path)[:20000]  # 처음 2만자만 분석
            else:
                content = read_file_content(file_path)
                
            # 내용이 있는 경우 분석
            if content:
                # AI 추출기 인스턴스 생성
                extractor = EnhancedComplianceExtractor(
                    api_key=api_key,
                    use_ngram=True,
                    use_ai=False  # API 키가 없으면 AI 사용 안함
                )
                
                # 문서 처리
                content_result = extractor.process_document(content)
        except Exception as e:
            log_debug(f"파일 크기 확인 또는 내용 분석 오류: {str(e)}")
        
        # 결과 통합
        combined_result = combine_file_results(
            filename=filename,
            content_result=content_result,
            filename_result=filename_info
        )
        
        # 파일 경로 추가
        combined_result['파일경로'] = str(file_path.parent)
        
        # 파일 인덱스 안전하게 추가
        if target_files is not None:
            try:
                combined_result['파일인덱스'] = target_files.index(file_path) + 1 if file_path in target_files else -1
            except:
                combined_result['파일인덱스'] = -1
        else:
            combined_result['파일인덱스'] = -1
        
        return combined_result
        
    except Exception as e:
        log_debug(f"파일 분석 중 오류: {str(e)}")
        log_debug(traceback.format_exc())
        # 오류 발생 시 기본 정보만 반환
        return {
            '파일명': file_path.name,
            '상태': '상태 불명',
            '만료일': '알 수 없음',
            '남은 일수': None,
            '파일명_준법감시필': None, 
            '파일명_유효기간': None,
            '파일명_유효기간_상태': '정보 없음',
            '파일명_남은일수': None,
            '파일내_준법감시필_번호': None,
            '파일내_유효기간_날짜': None,
            '파일내_유효기간_상태': '정보 없음',
            '파일내_남은일수': None,
            '파일경로': str(file_path.parent),
            '파일인덱스': -1
        }

def combine_file_results(filename, content_result, filename_result):
    """파일명과 내용에서 추출한 결과 통합"""
    # 기본값 설정
    result = {
        '파일명': filename,
        '상태': '상태 불명',
        '만료일': '알 수 없음',
        '남은 일수': None,
        
        # 파일명 정보
        '파일명_준법감시필': filename_result.get('compliance_number'),
        '파일명_유효기간': filename_result.get('expiry_date'),
        '파일명_유효기간_상태': filename_result.get('status', '정보 없음'),
        '파일명_남은일수': filename_result.get('days_to_expiry'),
        
        # 파일 내용 정보
        '파일내_준법감시필_번호': content_result.get('compliance_number'),
        '파일내_유효기간_날짜': content_result.get('expiry_date'),
        '파일내_유효기간_상태': '정보 없음',
        '파일내_남은일수': None,
        
        '신뢰도': content_result.get('confidence', 0.0),
        '파일경로': None
    }
    
    # 파일 내용에서 유효기간 상태 계산
    content_expiry = content_result.get('expiry_date')
    if content_expiry:
        try:
            today = datetime.now().date()
            content_date_obj = datetime.strptime(content_expiry, '%Y-%m-%d').date()
            content_days_left = (content_date_obj - today).days
            
            result['파일내_남은일수'] = content_days_left
            
            if content_days_left < 0:
                result['파일내_유효기간_상태'] = '만료됨'
            elif content_days_left <= 30:
                result['파일내_유효기간_상태'] = '30일 이내 만료'
            else:
                result['파일내_유효기간_상태'] = '유효함'
        except Exception as e:
            log_debug(f"내용 유효기간 상태 계산 오류: {e}")
    
    # 최종 만료일 및 상태 결정 (파일명 우선, 내용 차선)
    if filename_result.get('expiry_date'):
        result['만료일'] = filename_result.get('expiry_date')
        result['상태'] = filename_result.get('status')
        result['남은 일수'] = filename_result.get('days_to_expiry')
    elif content_expiry:
        result['만료일'] = content_expiry
        result['상태'] = result['파일내_유효기간_상태']
        result['남은 일수'] = result['파일내_남은일수']
    
    return result

def save_interim_results(results, batch_idx):
    """중간 결과 저장 함수"""
    try:
        interim_df = pd.DataFrame(results)
        interim_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        interim_file = f"interim_results_{batch_idx}_{interim_timestamp}.csv"
        interim_df.to_csv(interim_file, index=False, encoding='utf-8-sig')
        print(f"중간 결과 저장됨: {interim_file}")
        return interim_file
    except Exception as e:
        print(f"중간 결과 저장 중 오류: {e}")
        return None

def save_formatted_excel(df, excel_file):
    """결과를 서식이 적용된 Excel로 저장"""
    try:
        # ExcelWriter 사용
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            # DataFrame을 Excel로 변환
            df.to_excel(writer, index=False, sheet_name='준법감시')
            
            # 워크북과 워크시트 가져오기
            workbook = writer.book
            worksheet = writer.sheets['준법감시']
            
            # 열 너비 자동 조정
            for i, column in enumerate(df.columns):
                # 열 이름의 길이 확인
                column_width = max(len(str(column)), 10)  # 최소 너비 10
                
                # 열의 데이터 중 가장 긴 값의 길이 확인 (최대 100행까지만 확인)
                max_length = 0
                for j in range(min(len(df), 100)):  # 처리 시간 단축을 위해 최대 100행까지만 확인
                    cell_value = str(df.iloc[j, i])
                    max_length = max(max_length, len(cell_value))
                
                # 최종 열 너비 결정 (최대 100자, 기본 폰트 기준)
                column_width = min(max(column_width, max_length + 2), 100)  # +2는 여유 공간
                
                # 열 너비 설정
                col_letter = get_column_letter(i + 1)
                worksheet.column_dimensions[col_letter].width = column_width
            
            # 헤더 행 스타일 설정
            header_font = Font(bold=True)
            header_fill = PatternFill(start_color='E6E6E6', end_color='E6E6E6', fill_type='solid')
            
            for cell in worksheet[1]:
                cell.font = header_font
                cell.fill = header_fill
            
            # 데이터 행에 조건부 서식 설정 (상태에 따른 색상)
            status_colors = {
                '만료됨': 'FFC7CE',  # 밝은 빨강
                '30일 이내 만료': 'FFEB9C',  # 밝은 노랑
                '유효함': 'C6EFCE',  # 밝은 녹색
                '상태 불명': 'DDDDDD'  # 회색
            }
            
            # 상태 열 인덱스 찾기
            status_col_idx = df.columns.get_loc('상태') + 1  # Excel은 1부터 시작
            status_col_letter = get_column_letter(status_col_idx)
            
            # 각 상태별로 조건부 서식 설정
            for status, color in status_colors.items():
                rule = CellIsRule(
                    operator='equal',
                    formula=[f'"{status}"'],
                    stopIfTrue=True,
                    fill=PatternFill(start_color=color, end_color=color, fill_type='solid')
                )
                
                # 조건부 서식 적용 범위 (상태 열만)
                cell_range = f'{status_col_letter}2:{status_col_letter}{len(df) + 1}'
                worksheet.conditional_formatting.add(cell_range, rule)
                
        print(f"Excel 파일이 자동 열 너비 조정과 서식을 적용하여 저장되었습니다: {excel_file}")
        return True
    except Exception as e:
        print(f"Excel 저장 오류: {e}")
        return False

6. 텍스트 비교 알고리즘 (카피킬러 스타일)

In [ ]:
class EnhancedTextComparison:
    def __init__(self, min_ngram_size=3, max_ngram_size=5, threshold=0.6):
        self.min_ngram_size = min_ngram_size
        self.max_ngram_size = max_ngram_size
        self.threshold = threshold
        
    def _create_ngrams(self, text, n):
        """텍스트에서 n-gram 생성"""
        tokens = nltk.word_tokenize(text.lower())
        return list(ngrams(tokens, n))
    
    def _create_fingerprints(self, text):
        """텍스트에서 지문(fingerprint) 생성"""
        fingerprints = set()
        for n in range(self.min_ngram_size, self.max_ngram_size + 1):
            ngram_list = self._create_ngrams(text, n)
            for gram in ngram_list:
                # n-gram을 문자열로 변환하고 해시 생성
                gram_str = ' '.join(gram)
                fingerprint = hashlib.md5(gram_str.encode()).hexdigest()
                fingerprints.add(fingerprint)
        return fingerprints
    
    def _calculate_jaccard_similarity(self, set1, set2):
        """두 집합의 자카드 유사도 계산"""
        if not set1 or not set2:
            return 0.0
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        return intersection / union if union > 0 else 0.0
    
    def calculate_similarity(self, text1, text2):
        """두 텍스트 간 유사도 계산 (지문 기반)"""
        fingerprints1 = self._create_fingerprints(text1)
        fingerprints2 = self._create_fingerprints(text2)
        
        return self._calculate_jaccard_similarity(fingerprints1, fingerprints2)
    
    def find_similar_segments(self, text1, text2, window_size=100, step_size=50):
        """텍스트 내에서 유사한 세그먼트 찾기"""
        similar_segments = []
        
        # 텍스트를 세그먼트로 분할
        segments1 = [text1[i:i+window_size] for i in range(0, len(text1), step_size) if i+window_size <= len(text1)]
        segments2 = [text2[i:i+window_size] for i in range(0, len(text2), step_size) if i+window_size <= len(text2)]
        
        for i, seg1 in enumerate(segments1):
            for j, seg2 in enumerate(segments2):
                similarity = self.calculate_similarity(seg1, seg2)
                if similarity >= self.threshold:
                    similar_segments.append({
                        'segment1': seg1,
                        'segment2': seg2,
                        'position1': i * step_size,
                        'position2': j * step_size,
                        'similarity': similarity
                    })
        
        return similar_segments
    
    def find_similar_documents(self, documents, filenames):
        """문서 집합에서 유사한 문서 쌍 찾기"""
        similar_docs = []
        
        total_comparisons = len(documents) * (len(documents) - 1) // 2
        print(f"총 {total_comparisons}개의 문서 쌍을 비교합니다...")
        
        comparison_count = 0
        for i in range(len(documents)):
            for j in range(i+1, len(documents)):
                similarity = self.calculate_similarity(documents[i], documents[j])
                
                comparison_count += 1
                if comparison_count % 100 == 0 or comparison_count == total_comparisons:
                    progress = 100 * comparison_count / total_comparisons
                    print(f"진행 상황: {progress:.1f}% ({comparison_count}/{total_comparisons})")
                
                if similarity >= self.threshold:
                    similar_docs.append({
                        'doc1': filenames[i],
                        'doc2': filenames[j],
                        'similarity': similarity
                    })
        
        print(f"유사도 {self.threshold} 이상인 문서 쌍: {len(similar_docs)}개")
        return similar_docs

#7. 유효기간 분석 및 예측 클래스

In [ ]:
class ExpiryAnalyzer:
    def __init__(self):
        self.today = datetime.now()
        
    def predict_expiry_date(self, dates, approval_numbers, validity_sentences):
        """다양한 정보를 종합하여 유효기간 예측"""
        potential_expiry_dates = []
        
        # 1. 명시적 유효기간 또는 만료일이 있는 경우
        for date_info in dates:
            if date_info['date_type'] == '유효기간' or date_info['date_type'] == '만료일':
                if 'date' in date_info:  # 단일 날짜
                    potential_expiry_dates.append({
                        'date': date_info['date'],
                        'confidence': 0.9,
                        'source': '명시적 유효기간/만료일',
                        'context': date_info['context']
                    })
                elif 'end_date' in date_info:  # 날짜 범위
                    potential_expiry_dates.append({
                        'date': date_info['end_date'],
                        'confidence': 0.8,
                        'source': '유효기간 범위 종료일',
                        'context': date_info['context']
                    })
        
        # 2. 유효기간 문장에 포함된 날짜 활용
        for sentence in validity_sentences:
            for date_info in dates:
                if 'date' in date_info and date_info['context'] in sentence:
                    if date_info['date'] > self.today:  # 미래 날짜만 고려
                        potential_expiry_dates.append({
                            'date': date_info['date'],
                            'confidence': 0.7,
                            'source': '유효기간 문장 내 날짜',
                            'context': sentence
                        })
        
        # 3. 준법감시/심의필 번호 기준 추정 (통상 1년 유효기간 가정)
        for approval in approval_numbers:
            try:
                approval_year = int(approval['year'])
                # 승인일을 해당 연도의 1월 1일로 가정하고 1년 유효기간 추정
                approval_date = datetime(approval_year, 1, 1)
                estimated_expiry = approval_date + timedelta(days=365)
                
                potential_expiry_dates.append({
                    'date': estimated_expiry,
                    'confidence': 0.5,
                    'source': '심의필 번호 기반 추정',
                    'context': approval['full_text']
                })
            except:
                continue
        
        # 4. 일반 날짜 중 미래 날짜 고려 (낮은 신뢰도)
        future_dates = [date_info for date_info in dates 
                       if 'date' in date_info and date_info['date'] > self.today]
        
        for date_info in future_dates:
            potential_expiry_dates.append({
                'date': date_info['date'],
                'confidence': 0.3,
                'source': '문서 내 미래 날짜',
                'context': date_info['context']
            })
        
        # 결과가 없으면 None 반환
        if not potential_expiry_dates:
            return None
            
        # 신뢰도 기준 정렬
        potential_expiry_dates.sort(key=lambda x: x['confidence'], reverse=True)
        
        return potential_expiry_dates[0]  # 가장 신뢰도 높은 유효기간 반환
    
    def analyze_expiry_status(self, compliance_results):
        """추출된 정보를 기반으로 유효기간 상태 분석"""
        expiry_status = []
        
        for doc_result in tqdm(compliance_results, desc="유효기간 분석 중"):
            # 유효기간 예측
            expiry_prediction = self.predict_expiry_date(
                doc_result['dates'], 
                doc_result['approval_numbers'],
                doc_result['validity_sentences']
            )
            
            if expiry_prediction:
                expiry_date = expiry_prediction['date']
                days_to_expiry = (expiry_date - self.today).days
                
                if days_to_expiry < 0:
                    status = 'expired'
                elif days_to_expiry <= 30:
                    status = 'expiring_soon'
                else:
                    status = 'valid'
                    
                expiry_status.append({
                    'filename': doc_result['filename'],
                    'expiry_date': expiry_date,
                    'days_to_expiry': days_to_expiry,
                    'confidence': expiry_prediction['confidence'],
                    'source': expiry_prediction['source'],
                    'context': expiry_prediction['context'],
                    'status': status
                })
            else:
                expiry_status.append({
                    'filename': doc_result['filename'],
                    'expiry_date': None,
                    'days_to_expiry': None,
                    'confidence': 0,
                    'source': '정보 없음',
                    'context': '',
                    'status': 'unknown'
                })
        
        # 요약 통계
        status_counts = {'expired': 0, 'expiring_soon': 0, 'valid': 0, 'unknown': 0}
        for result in expiry_status:
            status_counts[result['status']] += 1
            
        print("\n유효기간 상태 분석 결과:")
        print(f"- 만료됨: {status_counts['expired']}개 ({100*status_counts['expired']/len(expiry_status):.1f}%)")
        print(f"- 30일 이내 만료 예정: {status_counts['expiring_soon']}개 ({100*status_counts['expiring_soon']/len(expiry_status):.1f}%)")
        print(f"- 유효함: {status_counts['valid']}개 ({100*status_counts['valid']/len(expiry_status):.1f}%)")
        print(f"- 상태 불명: {status_counts['unknown']}개 ({100*status_counts['unknown']/len(expiry_status):.1f}%)")
        
        return expiry_status

#8. 유사도 기반 정보 전파 클래스

In [ ]:
class SimilarityBasedInfoPropagation:
    def __init__(self, similarity_threshold=0.7):
        self.text_comparator = EnhancedTextComparison(threshold=similarity_threshold)
        
    def propagate_expiry_info(self, documents, filenames, expiry_status, file_contents):
        """유사 문서 간 유효기간 정보 전파"""
        # 유사 문서 찾기
        similar_docs = self.text_comparator.find_similar_documents(documents, filenames)
        
        # 불확실한 문서에 대해 유사 문서의 정보 활용
        unknown_docs = [status for status in expiry_status if status['status'] == 'unknown']
        
        updated_status = expiry_status.copy()
        
        if len(unknown_docs) > 0 and len(similar_docs) > 0:
            print(f"\n유사 문서 간 정보 전파를 시작합니다...")
            print(f"- 상태 불명 문서: {len(unknown_docs)}개")
            print(f"- 유사 문서 쌍: {len(similar_docs)}개")
            
            propagation_count = 0
            for unknown in unknown_docs:
                unknown_idx = next(i for i, s in enumerate(updated_status) if s['filename'] == unknown['filename'])
                
                # 유사 문서 중 유효기간 정보가 있는 문서 찾기
                for similar in similar_docs:
                    if similar['doc1'] == unknown['filename']:
                        similar_doc = similar['doc2']
                    elif similar['doc2'] == unknown['filename']:
                        similar_doc = similar['doc1']
                    else:
                        continue
                    
                    # 유사 문서의 유효기간 정보 가져오기
                    similar_status = next((s for s in expiry_status if s['filename'] == similar_doc and s['status'] != 'unknown'), None)
                    
                    if similar_status and similar_status['expiry_date']:
                        # 유사도에 따라 신뢰도 조정
                        confidence = similar_status['confidence'] * similar['similarity'] * 0.8
                        
                        updated_status[unknown_idx] = {
                            'filename': unknown['filename'],
                            'expiry_date': similar_status['expiry_date'],
                            'days_to_expiry': similar_status['days_to_expiry'],
                            'confidence': confidence,
                            'source': f"유사 문서({similar_doc})에서 전파",
                            'context': f"문서 유사도: {similar['similarity']:.2f}",
                            'status': similar_status['status']
                        }
                        propagation_count += 1
                        break
            
            print(f"정보 전파 완료: {propagation_count}개 문서에 유효기간 정보가 전파되었습니다.")
        
        # 최종 상태 통계
        status_counts = {'expired': 0, 'expiring_soon': 0, 'valid': 0, 'unknown': 0}
        for result in updated_status:
            status_counts[result['status']] += 1
            
        print("\n최종 유효기간 상태:")
        print(f"- 만료됨: {status_counts['expired']}개 ({100*status_counts['expired']/len(updated_status):.1f}%)")
        print(f"- 30일 이내 만료 예정: {status_counts['expiring_soon']}개 ({100*status_counts['expiring_soon']/len(updated_status):.1f}%)")
        print(f"- 유효함: {status_counts['valid']}개 ({100*status_counts['valid']/len(updated_status):.1f}%)")
        print(f"- 상태 불명: {status_counts['unknown']}개 ({100*status_counts['unknown']/len(updated_status):.1f}%)")
        
        return updated_status, similar_docs

#9. 결과 시각화 및 보고서 생성

In [ ]:
def visualize_and_report(expiry_status, similar_docs):
    """분석 결과 시각화 및 보고서 생성"""
    # 1. 유효기간 상태 차트
    status_counts = {'valid': 0, 'expiring_soon': 0, 'expired': 0, 'unknown': 0}
    for doc in expiry_status:
        status_counts[doc['status']] += 1
    
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    colors = ['green', 'orange', 'red', 'gray']
    labels = ['유효함', '30일 이내 만료', '만료됨', '상태 불명']
    values = [status_counts['valid'], status_counts['expiring_soon'], status_counts['expired'], status_counts['unknown']]
    
    plt.bar(labels, values, color=colors)
    plt.title('문서 유효기간 상태', fontsize=14)
    plt.ylabel('문서 수', fontsize=12)
    
    # 데이터 레이블 추가
    for i, v in enumerate(values):
        plt.text(i, v + 0.1, f"{v}개\n({100*v/sum(values):.1f}%)", 
                 ha='center', fontsize=10, fontweight='bold')
    
    # 2. 만료까지 남은 일수 분포
    plt.subplot(2, 2, 2)
    valid_docs = [doc for doc in expiry_status if doc['status'] in ['valid', 'expiring_soon']]
    if valid_docs:
        days_to_expiry = [doc['days_to_expiry'] for doc in valid_docs if doc['days_to_expiry'] is not None]
        if days_to_expiry:
            plt.hist(days_to_expiry, bins=10, color='blue', alpha=0.7)
            plt.title('만료까지 남은 일수 분포', fontsize=14)
            plt.xlabel('일수', fontsize=12)
            plt.ylabel('문서 수', fontsize=12)
    
    # 3. 신뢰도 분포
    plt.subplot(2, 2, 3)
    confidence_values = [doc['confidence'] for doc in expiry_status if doc['confidence'] > 0]
    if confidence_values:
        plt.hist(confidence_values, bins=10, color='purple', alpha=0.7)
        plt.title('유효기간 예측 신뢰도 분포', fontsize=14)
        plt.xlabel('신뢰도', fontsize=12)
        plt.ylabel('문서 수', fontsize=12)
    
    # 4. 유사도 분포
    plt.subplot(2, 2, 4)
    if similar_docs:
        similarities = [doc['similarity'] for doc in similar_docs]
        plt.hist(similarities, bins=10, color='green', alpha=0.7)
        plt.title('문서 간 유사도 분포', fontsize=14)
        plt.xlabel('유사도', fontsize=12)
        plt.ylabel('문서 쌍 수', fontsize=12)
    
    plt.tight_layout()
    plt.savefig(str(BASE_PATH / 'compliance_analysis_report.png'))
    plt.show()
    
    # 텍스트 보고서 생성
    print("\n=== IBK 준법감시 문서 분석 보고서 ===")
    
    print("\n[유효기간 만료 문서]")
    expired_docs = [doc for doc in expiry_status if doc['status'] == 'expired']
    for doc in expired_docs[:10]:  # 상위 10개만 표시
        print(f"- {doc['filename']} (만료일: {doc['expiry_date'].strftime('%Y-%m-%d') if doc['expiry_date'] else '알 수 없음'}, 신뢰도: {doc['confidence']:.2f})")
        print(f"  근거: {doc['source']}")
        print(f"  컨텍스트: {doc['context']}")
    
    if len(expired_docs) > 10:
        print(f"  ...외 {len(expired_docs)-10}개 더 있음")
    
    print("\n[30일 이내 만료 예정 문서]")
    expiring_soon = [doc for doc in expiry_status if doc['status'] == 'expiring_soon']
    for doc in expiring_soon[:10]:  # 상위 10개만 표시
        print(f"- {doc['filename']} (만료일: {doc['expiry_date'].strftime('%Y-%m-%d')}, 남은 일수: {doc['days_to_expiry']}일, 신뢰도: {doc['confidence']:.2f})")
        print(f"  근거: {doc['source']}")
        print(f"  컨텍스트: {doc['context']}")
    
    if len(expiring_soon) > 10:
        print(f"  ...외 {len(expiring_soon)-10}개 더 있음")
    
    print("\n[유사도 높은 문서 쌍]")
    # 유사도가 높은 순으로 정렬
    sorted_similar_docs = sorted(similar_docs, key=lambda x: x['similarity'], reverse=True)
    for pair in sorted_similar_docs[:10]:  # 상위 10개만 표시
        print(f"- {pair['doc1']} <-> {pair['doc2']} (유사도: {pair['similarity']:.2f})")
    
    if len(sorted_similar_docs) > 10:
        print(f"  ...외 {len(sorted_similar_docs)-10}개 더 있음")
    
    # DataFrame 반환
    df_expiry = pd.DataFrame(expiry_status)
    df_similarity = pd.DataFrame(similar_docs) if similar_docs else pd.DataFrame()
    
    # 결과 저장
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    df_expiry.to_csv(str(BASE_PATH / f'ibk_compliance_expiry_report_{timestamp}.csv'), index=False, encoding='utf-8-sig')
    if not df_similarity.empty:
        df_similarity.to_csv(str(BASE_PATH / f'ibk_document_similarity_report_{timestamp}.csv'), index=False, encoding='utf-8-sig')
    
    return df_expiry, df_similarity

#10. 필요한 환경 준비 기능

In [ ]:
def prepare_environment():
    """필요한 라이브러리 설치 확인"""
    required_libraries = {
        'PyPDF2': ('PyPDF2', 'PDF 파일 처리'),
        'python-docx': ('docx', 'Word 문서 처리'),
        'openpyxl': ('openpyxl', 'Excel 파일 처리'),
        'python-pptx': ('pptx', 'PowerPoint 파일 처리'),
        'olefile': ('olefile', 'HWP 파일 헤더 처리')
    }
    
    installed = []
    missing = []
    
    for display_name, (import_name, purpose) in required_libraries.items():
        try:
            __import__(import_name)
            installed.append(f"✓ {display_name} 설치됨 ({purpose})")
        except ImportError:
            missing.append(f"✗ {display_name} 미설치 ({purpose})")
    
    # 결과 출력
    for msg in installed:
        print(msg)
    for msg in missing:
        print(msg)
    
    if missing:
        print("\n다음 라이브러리 설치가 필요합니다:")
        for lib, (_, _) in [(k, v) for k, v in required_libraries.items() if f"✗ {k}" in " ".join(missing)]:
            print(f"pip install {lib}")
    
    return len(missing) == 0

#11.IBK 준법감시 문서 분석 파이프라인

In [ ]:
# 전역 변수로 선언
problematic_pdfs = set()
updated_expiry_status = []  

def process_pdf_safely(file_path):
    """안전하게 PDF 처리 - PyPDF2만 사용"""
    if file_path in problematic_pdfs:
        print(f"이전에 문제가 있던 PDF 건너뜀: {file_path.name}")
        return ""
    
    content = ""
    try:
        import PyPDF2
        try:
            with open(file_path, 'rb') as file:
                try:
                    # strict=False로 설정하여 일부 오류 무시
                    reader = PyPDF2.PdfReader(file, strict=False)
                    
                    # 페이지별로 처리하여 한 페이지에서 오류가 발생해도 계속 진행
                    for page_num in range(len(reader.pages)):
                        try:
                            page_text = reader.pages[page_num].extract_text()
                            if page_text:
                                content += page_text + "\n"
                        except Exception as e:
                            # 특정 페이지 처리 오류는 기록만 하고 계속 진행
                            print(f"페이지 {page_num} 처리 오류({file_path.name}): {type(e).__name__}")
                            continue
                
                except Exception as e:
                    print(f"PDF 파일 구조 오류({file_path.name}): {type(e).__name__}")
                    # 오류가 있는 PDF 파일 목록에 추가
                    problematic_pdfs.add(file_path)
        except Exception as e:
            print(f"파일 열기 오류({file_path.name}): {type(e).__name__}")
            problematic_pdfs.add(file_path)
    except ImportError:
        print("PyPDF2 라이브러리가 필요합니다.")
    
    return content

def extract_expiry_date_from_filename(filename):
    """파일명에서 유효기간 정보를 추출"""
    # 정규표현식을 사용하여 유효기간 패턴 찾기
    # 예: '20240531', '2024-05-31', '유효기간_20240531' 등
    
    import re
    from datetime import datetime
    
    # 다양한 날짜 패턴 처리
    patterns = [
        r'(\d{4})[년\-_.]?(\d{1,2})[월\-_.]?(\d{1,2})[일]?',  # 2024년05월31일, 2024-05-31, 2024_05_31
        r'유효기간[_\s]?(\d{4})[년\-_.]?(\d{1,2})[월\-_.]?(\d{1,2})[일]?',  # 유효기간_20240531
        r'expiry[_\s]?(\d{4})[년\-_.]?(\d{1,2})[월\-_.]?(\d{1,2})[일]?'  # expiry_20240531
    ]
    
    for pattern in patterns:
        match = re.search(pattern, filename)
        if match:
            try:
                year = int(match.group(1))
                month = int(match.group(2))
                day = int(match.group(3))
                
                # 유효한 날짜인지 확인
                if 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                    return datetime(year, month, day)
            except (ValueError, IndexError):
                continue
    
    return None

def visualize_and_report(expiry_status):
    """결과 시각화 및 보고서 생성 함수"""
    import pandas as pd
    
    # 유효기간 상태를 데이터프레임으로 변환
    if expiry_status and len(expiry_status) > 0:
        df_expiry = pd.DataFrame(expiry_status)
        
        # 날짜 형식 변환 및 정렬
        if 'expiry_date' in df_expiry.columns and df_expiry['expiry_date'].notna().any():
            df_expiry['expiry_date'] = pd.to_datetime(df_expiry['expiry_date'])
            df_expiry = df_expiry.sort_values(by='days_to_expiry')
        
        # 상태별 개수 통계
        status_counts = df_expiry['status'].value_counts()
        print("\n=== 문서 상태 통계 ===")
        print(f"만료됨: {status_counts.get('expired', 0)}개")
        print(f"곧 만료됨(30일 이내): {status_counts.get('expiring_soon', 0)}개")
        print(f"유효함: {status_counts.get('valid', 0)}개")
        print(f"상태 알 수 없음: {status_counts.get('unknown', 0)}개")
    else:
        df_expiry = pd.DataFrame(columns=['filename', 'expiry_date', 'days_to_expiry', 'status', 'confidence', 'source', 'context', 'has_compliance_stamp'])
    
    # 빈 유사도 데이터프레임 반환 (유사문서 분석 기능 제외)
    df_similarity = pd.DataFrame()
    
    return df_expiry, df_similarity

def load_documents(directory_path):
    """문서를 로드하고 전처리하는 함수"""
    documents = []
    filenames = []
    file_contents = {}
    filename_expiry_dates = {}
    ext = file_path.suffix.lower()

    # 파일 목록 수집
    all_files = list(Path(directory_path).rglob('*'))
    print(f"파일 목록 수집 중...")
    
    # 우선 처리해야 할 파일들 (문서 파일)
    priority_extensions = ['.pdf', '.docx', '.doc', '.pptx', '.ppt', '.xlsx', '.xls', '.hwp']
    priority_files = [f for f in all_files if f.suffix.lower() in priority_extensions]
    other_files = [f for f in all_files if f.suffix.lower() not in priority_extensions]
    
    print(f"우선 처리할 파일: {len(priority_files)}개")
    print(f"일반 처리할 파일: {len(other_files)}개")
    
    # 파일명에서 유효기간 정보 추출
    for file_path in priority_files + other_files:
        expiry_date = extract_expiry_date_from_filename(file_path.name)
        if expiry_date:
            filename_expiry_dates[file_path.name] = expiry_date
    
    print(f"파일명에서 유효기간 정보 추출: {len(filename_expiry_dates)}개")
    
    # 진행 상황 추적
    processed_count = 0
    success_count = 0
    
    # 문서 파일 처리
    for file_path in priority_files:
        processed_count += 1
        
        # 진행 상황 출력 (100개마다)
        if processed_count % 100 == 0:
            print(f"처리 중... {processed_count}/{len(priority_files)} 파일 ({success_count} 성공)")
        
        try:
            content = ""
            
            # PDF 파일 처리
            if file_path.suffix.lower() == '.pdf':
                content = process_pdf_safely(file_path)
            
            # Word 문서 처리
            if ext == '.docx':
                try:
                    import docx
                    doc = docx.Document(file_path)
                    
                    # 문단 텍스트 추출
                    paragraphs_text = []
                    for para in doc.paragraphs:
                        if para.text.strip():
                            paragraphs_text.append(para.text)
                    
                    # 테이블 텍스트 추출 추가
                    tables_text = []
                    for table in doc.tables:
                        for row in table.rows:
                            row_text = []
                            for cell in row.cells:
                                if cell.text.strip():
                                    row_text.append(cell.text)
                            if row_text:
                                tables_text.append(' | '.join(row_text))
                    
                    # 모든 텍스트 결합
                    all_texts = paragraphs_text + tables_text
                    content = "\n".join(all_texts)
                    
                    # 디버깅
                    print(f"DOCX 처리: {len(all_texts)}개 텍스트 블록 추출됨")
                except Exception as e:
                    print(f"DOCX 처리 오류: {str(e)}")
                    content = ""
            
            # PowerPoint 파일 처리
            elif file_path.suffix.lower() == '.pptx':
                try:
                    from pptx import Presentation
                    prs = Presentation(file_path)
                    for slide in prs.slides:
                        for shape in slide.shapes:
                            if hasattr(shape, "text"):
                                content += shape.text + "\n"
                except ImportError:
                    print("python-pptx 라이브러리가 필요합니다.")
                except Exception as e:
                    print(f"PowerPoint 문서 처리 오류({file_path.name}): {type(e).__name__}")
            
            # Excel 파일 처리
            elif file_path.suffix.lower() in ['.xlsx', '.xls']:
                try:
                    import pandas as pd
                    # 모든 시트 읽기
                    sheets = pd.read_excel(file_path, sheet_name=None)
                    for sheet_name, df in sheets.items():
                        content += f"시트: {sheet_name}\n"
                        content += df.to_string(index=False) + "\n\n"
                except ImportError:
                    print("pandas와 openpyxl 라이브러리가 필요합니다.")
                except Exception as e:
                    print(f"Excel 문서 처리 오류({file_path.name}): {type(e).__name__}")
            
            # 내용이 있으면 추가
            if content and content.strip():
                documents.append(content)
                filenames.append(file_path.name)
                file_contents[file_path.name] = content
                success_count += 1
        
        except Exception as e:
            print(f"파일 처리 중 오류 발생({file_path.name}): {type(e).__name__}")
            continue
    
    print(f"처리 완료: 총 {len(priority_files)}개 파일 중 {success_count}개 성공")
    return documents, filenames, file_contents, filename_expiry_dates

def analyze_ibk_compliance_documents(directory_path, similarity_threshold=0.7):
    """IBK 준법감시 문서 분석 파이프라인"""
    global updated_expiry_status
    
    print("IBK 준법감시 문서 분석을 시작합니다...")
    
    # 1. 문서 로드 및 전처리 (수정된 함수 사용)
    documents, filenames, file_contents, filename_expiry_dates = load_documents(directory_path)
    
    if not documents or len(documents) == 0:
        print("분석할 문서가 없습니다.")
        return None, None
    
    # 2. 준법감시 정보 추출
    extractor = IBKComplianceExtractor()
    compliance_results = extractor.extract_compliance_info(documents, filenames)
    
    if not compliance_results or len(compliance_results) == 0:
        print("준법감시 정보를 추출할 수 없습니다.")
        return None, None
        
    print("준법감시 정보 추출을 완료했습니다.")
    
    # 3. 유효기간 분석
    analyzer = ExpiryAnalyzer()
    expiry_status = analyzer.analyze_expiry_status(compliance_results)
    print("유효기간 분석을 완료했습니다.")
    
    # 3.1 파일명 기반 유효기간 정보 추가 (새로 추가)
    enhanced_expiry_status = []
    for status in expiry_status:
        # 파일명에서 추출한 유효기간 정보가 있으면 활용
        if status['filename'] in filename_expiry_dates:
            # 문서 내용에서 추출한 정보가 없거나 신뢰도가 낮은 경우
            if status['status'] == 'unknown' or status['confidence'] < 0.5:
                expiry_date = filename_expiry_dates[status['filename']]
                days_to_expiry = (expiry_date - datetime.now()).days
                
                if days_to_expiry < 0:
                    new_status = 'expired'
                elif days_to_expiry <= 30:
                    new_status = 'expiring_soon'
                else:
                    new_status = 'valid'
                
                enhanced_status = {
                    'filename': status['filename'],
                    'expiry_date': expiry_date,
                    'days_to_expiry': days_to_expiry,
                    'confidence': 0.8,  # 파일명 기반은 높은 신뢰도 부여
                    'source': '파일명에서 추출',
                    'context': f'파일명: {status["filename"]}',
                    'status': new_status,
                    'has_compliance_stamp': status.get('has_compliance_stamp', False)  # 준법감사필 여부 유지
                }
                enhanced_expiry_status.append(enhanced_status)
            else:
                enhanced_expiry_status.append(status)
        else:
            enhanced_expiry_status.append(status)
    
    print(f"파일명 기반 유효기간 정보 추가: {len(filename_expiry_dates)}개")
    
    # enhanced_expiry_status를 updated_expiry_status로 복사
    updated_expiry_status = enhanced_expiry_status.copy()
    
    # 4. 결과 시각화 및 보고서 생성 (유사 문서 분석 제외)
    df_expiry, df_similarity = visualize_and_report(updated_expiry_status)
    print("결과 시각화 및 보고서 생성을 완료했습니다.")
    
    # 결과를 데이터프레임 형태로 반환
    return df_expiry, df_similarity

# IBKComplianceExtractor 클래스 - 준법감사필 문구 확인에 초점
class IBKComplianceExtractor:
    def extract_compliance_info(self, documents, filenames):
        """문서에서 준법감시 정보 추출"""
        results = []
        
        # 준법감사필 관련 키워드
        compliance_keywords = [
            '준법감사필', '준법감시필', '준법심사필', '준법검토필', 
            '준법감사', '준법감시', '준법심사', '준법검토',
            '법무감사', '법무검토', '법무심사'
        ]
        
        for doc, filename in zip(documents, filenames):
            # 준법감사필 여부 확인
            has_compliance_stamp = any(keyword in doc for keyword in compliance_keywords)
            
            result = {
                'filename': filename,
                'content': doc[:1000],  # 내용 일부만 저장 (분석용)
                'has_compliance_stamp': has_compliance_stamp
            }
            results.append(result)
            
        # 준법감사필 있는 문서 통계
        compliant_docs = sum(1 for r in results if r['has_compliance_stamp'])
        print(f"준법감사필 포함 문서: {compliant_docs}개 / 전체 {len(results)}개 ({(compliant_docs/len(results)*100 if len(results) > 0 else 0):.1f}%)")
        
        return results

# ExpiryAnalyzer 클래스 - 유효기간 확인에 초점
class ExpiryAnalyzer:
    def analyze_expiry_status(self, compliance_results):
        """준법감시 문서의 유효기간 분석"""
        from datetime import datetime
        import re
        
        expiry_statuses = []
        
        for result in compliance_results:
            filename = result['filename']
            content = result.get('content', '')
            has_compliance_stamp = result.get('has_compliance_stamp', False)
            
            # 유효기간 패턴 추출
            expiry_date = self._extract_expiry_date(content)
            
            if expiry_date:
                days_to_expiry = (expiry_date - datetime.now()).days
                
                if days_to_expiry < 0:
                    status = 'expired'
                elif days_to_expiry <= 30:
                    status = 'expiring_soon'
                else:
                    status = 'valid'
                
                # 준법감사필이 있으면 신뢰도 증가
                confidence = 0.8 if has_compliance_stamp else 0.6
                
                expiry_statuses.append({
                    'filename': filename,
                    'expiry_date': expiry_date,
                    'days_to_expiry': days_to_expiry,
                    'status': status,
                    'confidence': confidence,
                    'source': '문서 내용에서 추출',
                    'context': self._get_context(content, expiry_date),
                    'has_compliance_stamp': has_compliance_stamp
                })
            else:
                expiry_statuses.append({
                    'filename': filename,
                    'expiry_date': None,
                    'days_to_expiry': None,
                    'status': 'unknown',
                    'confidence': 0.0,
                    'source': '정보 없음',
                    'context': '유효기간 정보를 찾을 수 없습니다.',
                    'has_compliance_stamp': has_compliance_stamp
                })
        
        return expiry_statuses
    
    def _extract_expiry_date(self, text):
        """텍스트에서 유효기간 날짜 추출"""
        from datetime import datetime
        import re
        
        # 다양한 유효기간 패턴 처리
        patterns = [
            r'유효기간[:\s]*(\d{4})[년\-/]?(\d{1,2})[월\-/]?(\d{1,2})일?',
            r'만료일[:\s]*(\d{4})[년\-/]?(\d{1,2})[월\-/]?(\d{1,2})일?',
            r'효력기간[:\s]*~\s*(\d{4})[년\-/]?(\d{1,2})[월\-/]?(\d{1,2})일?',
            r'(\d{4})[년\-/]?(\d{1,2})[월\-/]?(\d{1,2})일?까지\s*유효',
            r'(\d{4})[년\-/]?(\d{1,2})[월\-/]?(\d{1,2})일?까지',
            r'(\d{4})[년\-/]?(\d{1,2})[월\-/]?(\d{1,2})일?\s*만료',
            r'준법감사필\s*[\(\［].*?(\d{4})[년\-/]?(\d{1,2})[월\-/]?(\d{1,2})일?.*?[\)\］]'
        ]
        
        for pattern in patterns:
            matches = re.finditer(pattern, text)
            for match in matches:
                try:
                    year = int(match.group(1))
                    month = int(match.group(2))
                    day = int(match.group(3))
                    
                    # 유효한 날짜인지 확인
                    if 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                        return datetime(year, month, day)
                except (ValueError, IndexError):
                    continue
        
        return None
    
    def _get_context(self, text, expiry_date):
        """유효기간 정보가 포함된 문맥 추출"""
        import re
        
        # 유효기간과 관련된 문맥 패턴들
        context_patterns = [
            r'([^.!?]*유효기간[^.!?]*)',
            r'([^.!?]*만료일[^.!?]*)',
            r'([^.!?]*효력기간[^.!?]*)',
            r'([^.!?]*까지\s*유효[^.!?]*)',
            r'([^.!?]*준법감사필[^.!?]*)'
        ]
        
        formatted_date = expiry_date.strftime("%Y년 %m월 %d일")
        
        # 모든 패턴에 대해 시도
        for pattern in context_patterns:
            match = re.search(pattern, text)
            if match:
                context = match.group(1).strip()
                if len(context) > 10:  # 최소 길이 확인
                    return context
        
        # 특정 문맥을 찾지 못한 경우 기본값 반환
        return f"유효기간: {formatted_date}"

#12.결과 데이터프레임 생성 및 반환

In [ ]:
def create_result_dataframe(df_expiry):
    """분석 결과를 정리된 데이터프레임으로 변환"""
    if df_expiry is None or len(df_expiry) == 0:
        return pd.DataFrame()
    
    # 상태 한글 표현 매핑
    status_mapping = {
        'valid': '유효함',
        'expiring_soon': '30일 이내 만료',
        'expired': '만료됨',
        'unknown': '상태 불명'
    }
    
    # 필요한 열만 선택하고 정렬
    result_df = df_expiry.copy()
    
    # 상태 한글화
    result_df['상태'] = result_df['status'].map(status_mapping)
    
    # 날짜 형식 변환
    result_df['만료일'] = result_df['expiry_date'].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else '알 수 없음'
    )
    
    # 남은 일수 형식 변환
    result_df['남은_일수'] = result_df['days_to_expiry'].apply(
        lambda x: f"{x}일" if pd.notnull(x) else '알 수 없음'
    )
    
    # 신뢰도 백분율 표현
    result_df['신뢰도'] = result_df['confidence'].apply(
        lambda x: f"{x*100:.1f}%" if pd.notnull(x) else '0%'
    )
    
    # 열 재구성
    columns = {
        'filename': '파일명',
        '상태': '상태',
        '만료일': '만료일',
        '남은_일수': '남은 일수',
        '신뢰도': '신뢰도',
        'source': '정보 출처',
        'context': '컨텍스트'
    }
    
    result_df = result_df.rename(columns=columns)
    result_df = result_df[list(columns.values())]
    
    # 상태별 정렬
    status_order = ['만료됨', '30일 이내 만료', '유효함', '상태 불명']
    result_df['상태_순서'] = result_df['상태'].apply(lambda x: status_order.index(x))
    result_df = result_df.sort_values(['상태_순서', '만료일']).drop('상태_순서', axis=1)
    
    return result_df

#13. 메인 실행함수

In [ ]:
def process_documents_in_batches(file_paths, batch_size=50, max_workers=8):
    """파일 배치 처리 및 병렬화 - 메모리 효율성 개선"""
    import concurrent.futures
    import math
    from tqdm.notebook import tqdm
    
    # 배치로 나누기
    batches = [file_paths[i:i+batch_size] for i in range(0, len(file_paths), batch_size)]
    total_batches = len(batches)
    
    all_results = []
    extractor = EnhancedComplianceExtractor(
        api_key=api_key,
        use_ngram=True,
        use_ai=True
    )
    
    print(f"총 {len(file_paths)}개 파일을 {total_batches}개 배치로 처리합니다.")
    
    # 각 배치 처리
    for batch_idx, batch in enumerate(batches):
        print(f"\n배치 {batch_idx+1}/{total_batches} 처리 중 ({len(batch)}개 파일)...")
        batch_results = []
        
        # 병렬 처리 함수
        def process_file(file_path):
            try:
                # 파일 읽기
                content = read_file_content(file_path)
                
                if not content:
                    return {
                        "filename": file_path.name,
                        "status": "상태 불명",
                        "expiry_date": None,
                        "days_left": None,
                        "compliance_number": None,
                        "confidence": 0.0,
                        "error": "파일 내용 없음"
                    }
                
                # 문서 분석
                result = extractor.extract_compliance_info(content)
                
                # 파일명에서도 정보 추출
                filename_info = extract_info_from_filename(file_path.name)
                
                # 결과 통합
                combined_result = combine_file_results(
                    filename=file_path.name,
                    content_result=result,
                    filename_result=filename_info
                )
                
                return combined_result
                
            except Exception as e:
                return {
                    "filename": file_path.name,
                    "status": "오류",
                    "expiry_date": None,
                    "days_left": None,
                    "compliance_number": None,
                    "confidence": 0.0,
                    "error": str(e)
                }
        
        # 병렬 실행
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_file = {executor.submit(process_file, file_path): file_path for file_path in batch}
            
            for future in tqdm(concurrent.futures.as_completed(future_to_file), 
                               total=len(batch), 
                               desc=f"배치 {batch_idx+1} 처리"):
                file_path = future_to_file[future]
                try:
                    result = future.result()
                    batch_results.append(result)
                except Exception as e:
                    print(f"파일 {file_path.name} 처리 중 오류: {str(e)}")
        
        # 배치 결과 저장
        all_results.extend(batch_results)
        
        # 중간 저장
        #if batch_idx % 2 == 1 or batch_idx == total_batches - 1:
        #    save_interim_results(all_results, batch_idx)
    
    return all_results

In [ ]:
# 계층적 하이브리드 방식 적용 - 준법감시필 번호 및 유효기간 추출
try:
    from datetime import datetime
    import pandas as pd
    import re
    import os
    from pathlib import Path
    import time
    import traceback
    from tqdm.notebook import tqdm
    import gc
    import concurrent.futures
    import openpyxl  # 명시적으로 openpyxl 추가
    from openpyxl.styles import Font, PatternFill  # 스타일 관련 모듈도 추가
    from openpyxl.formatting.rule import CellIsRule  # 조건부 서식을 위한 모듈
    from openpyxl.utils import get_column_letter  # 열 문자 변환 유틸리티

    start_time = time.time()
    
    # 현재 디렉토리 확인
    current_dir = os.getcwd()
    print(f"현재 디렉토리: {current_dir}")
    base_path = Path(current_dir)
    
    # 오류 날짜 목록 - 이 날짜들은 검사에서 제외됩니다
    ERROR_DATES = ['2024-11-30', '2020-12-31']
    
    # 디버깅 모드 활성화
    DEBUG_MODE = True
    DEBUG_FILE = "debug_log.txt"
    
    # 디버깅 로그 함수
    def log_debug(message):
        if DEBUG_MODE:
            with open(DEBUG_FILE, "a", encoding="utf-8") as f:
                f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")
                
    # 디버그 로그 파일 초기화
    with open(DEBUG_FILE, "w", encoding="utf-8") as f:
        f.write(f"=== 디버그 로그 시작: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ===\n")
    
    class AIEnhancedComplianceExtractor:
        def __init__(self, api_key=None, use_ngram=True, use_ai=False):
            self.api_key = api_key
            self.use_ngram = use_ngram
            self.use_ai = use_ai
            
            # N-gram 관련 설정
            self.ngram_sizes = [2, 3, 4]  # 2-gram, 3-gram, 4-gram 사용
            
            # 키워드 설정
            self.compliance_keywords = [
                '준법감시', '준법감시인', '준법감사', '심의필', '제호', 
                '승인', '결재', '법규', '컴플라이언스'
            ]
            
            self.validity_keywords = [
                '유효기간', '만료일', '유효', '만료', '기간', 
                '까지', '효력', '사용기한'
            ]
        
        def extract_ngrams(self, text):
            """텍스트에서 n-gram 추출"""
            import re
            from collections import Counter
            
            # 텍스트 전처리 (특수문자 제거 및 공백 정규화)
            cleaned_text = re.sub(r'[^\w\s]', ' ', text)
            cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
            
            # 단어 목록 생성
            words = cleaned_text.split()
            
            # n-gram 생성
            all_ngrams = []
            
            for n in self.ngram_sizes:
                if len(words) < n:
                    continue
                    
                ngrams = [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]
                all_ngrams.extend(ngrams)
            
            # n-gram 빈도 계산
            ngram_freq = Counter(all_ngrams)
            
            return ngram_freq
            
        def score_text_for_compliance(self, text):
            """N-gram 기반 준법감시 관련 텍스트 점수 계산"""
            score = 0
            
            # 키워드 기반 점수 계산
            for keyword in self.compliance_keywords:
                if keyword in text:
                    score += 5  # 기본 키워드 발견 시 5점
                    
            for keyword in self.validity_keywords:
                if keyword in text:
                    score += 5  # 유효기간 키워드 발견 시 5점
            
            # N-gram 기반 점수 계산
            if self.use_ngram:
                ngrams = self.extract_ngrams(text)
                
                # N-gram에 키워드가 포함된 경우 점수 추가
                for ngram, freq in ngrams.items():
                    compliance_matches = sum(1 for keyword in self.compliance_keywords if keyword in ngram)
                    validity_matches = sum(1 for keyword in self.validity_keywords if keyword in ngram)
                    
                    score += compliance_matches * 2 * freq  # 준법 키워드 포함 n-gram당 2점
                    score += validity_matches * 2 * freq    # 유효기간 키워드 포함 n-gram당 2점
            
            return score
            
        def find_best_sections(self, text, window_size=200, step_size=100):
            """준법감시 관련 최적 섹션 찾기"""
            if len(text) <= window_size:
                return [text]
                
            # 텍스트를 섹션으로 분할
            sections = []
            for i in range(0, len(text) - window_size + 1, step_size):
                section = text[i:i+window_size]
                sections.append(section)
            
            # 각 섹션 점수 계산
            section_scores = [(section, self.score_text_for_compliance(section)) for section in sections]
            
            # 점수 기준 정렬
            section_scores.sort(key=lambda x: x[1], reverse=True)
            
            # 상위 5개 섹션 반환
            top_sections = [section for section, score in section_scores[:5]]
            
            return top_sections
        
        def extract_with_regex(self, text):
            """정규식으로 준법감시 정보 추출"""
            log_debug("정규식으로 준법감시 정보 추출 시작")
            result = {"compliance_number": None, "expiry_date": None}
            
            # 준법감시필 번호 추출
            compliance_patterns = [
                r'제(\d{4})-(\d+)호',  # 제2024-4806호
                r'준법감시[_\-\s]*(\d{4})[_\-\s]*(\d+)',  # 준법감시-2024-123
                r'심의필[_\-\s]*(\d{4})[_\-\s]*(\d+)',    # 심의필-2024-123
                r'준법[_\-\s]*(\d{4})[_\-\s]*(\d+)'       # 준법-2024-123
            ]
            
            for pattern in compliance_patterns:
                match = re.search(pattern, text)
                if match:
                    try:
                        year, number = match.groups()
                        
                        # 형식에 따라 준법감시필 번호 생성
                        if pattern.startswith(r'제'):
                            result["compliance_number"] = f"제{year}-{number}호"
                        elif pattern.startswith(r'준법감시'):
                            result["compliance_number"] = f"준법감시-{year}-{number}"
                        elif pattern.startswith(r'심의필'):
                            result["compliance_number"] = f"심의필-{year}-{number}"
                        else:
                            result["compliance_number"] = f"준법-{year}-{number}"
                        
                        log_debug(f"정규식으로 준법감시필 번호 추출: {result['compliance_number']}")
                        break
                    except:
                        continue
            
            # 유효기간 추출
            expiry_patterns = [
                r'유효기간[^\d]*(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?',  # 유효기간: 2025.08.20
                r'만료일[^\d]*(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?',   # 만료일: 2025.08.20
                r'(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?[^\d]*까지',  # 2025.08.20까지
                r'(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?[^\d]*만료',  # 2025.08.20 만료
                r'유효[^\d]*(\d{4})[-/.년\s](\d{1,2})[-/.월\s](\d{1,2})일?'   # 유효: 2025.08.20
            ]
            
            for pattern in expiry_patterns:
                match = re.search(pattern, text)
                if match:
                    try:
                        year, month, day = map(int, match.groups())
                        if 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                            date_str = f"{year}-{month:02d}-{day:02d}"
                            
                            # 오류 날짜 필터링
                            if date_str in ERROR_DATES:
                                log_debug(f"오류 날짜 감지: {date_str}")
                                continue
                            
                            result["expiry_date"] = date_str
                            log_debug(f"정규식으로 유효기간 추출: {date_str}")
                            break
                    except:
                        continue
                        
            return result
        
        def process_document(self, content):
            """문서 내용에서 준법감시필 번호와 유효기간 추출"""
            log_debug("문서 내용 분석 시작")
            
            # 기본 결과 초기화
            result = {"compliance_number": None, "expiry_date": None}
            
            # 1. 관련성 높은 섹션 찾기
            if len(content) > 1000:  # 긴 문서는 섹션으로 나누어 처리
                log_debug(f"긴 문서 감지: {len(content)} 자")
                sections = self.find_best_sections(content)
                log_debug(f"관련 섹션 {len(sections)}개 발견")
                
                # 각 섹션에서 정보 추출
                for section in sections:
                    section_result = self.extract_with_regex(section)
                    
                    # 결과 통합
                    if section_result["compliance_number"] and not result["compliance_number"]:
                        result["compliance_number"] = section_result["compliance_number"]
                        
                    if section_result["expiry_date"] and not result["expiry_date"]:
                        result["expiry_date"] = section_result["expiry_date"]
                        
                    # 모든 정보를 찾았으면 종료
                    if result["compliance_number"] and result["expiry_date"]:
                        break
            else:
                # 짧은 문서는 전체 분석
                result = self.extract_with_regex(content)
            
            # AI 분석 (사용 설정된 경우)
            if self.use_ai and self.api_key:
                try:
                    # 간단한 API 호출 구현 (실제 API 키가 없으므로 주석 처리)
                    # ai_result = self.call_ai_api(content)
                    # if ai_result:
                    #     if ai_result.get("compliance_number") and not result["compliance_number"]:
                    #         result["compliance_number"] = ai_result["compliance_number"]
                    #     if ai_result.get("expiry_date") and not result["expiry_date"]:
                    #         result["expiry_date"] = ai_result["expiry_date"]
                    pass
                except:
                    log_debug("AI 분석 중 오류 발생")
            
            log_debug(f"문서 내용 분석 결과: {result}")
            return result
    
    # 파일명에서 유효기간 추출 함수 강화
    def extract_expiry_date(filename):
        """파일명에서 유효기간을 추출하되, 오류 날짜는 필터링"""
        log_debug(f"파일명 분석 시작: {filename}")
        
        expiry_date = None
        expiry_patterns = [
            r'유효기간\((\d{4})\.(\d{2})\.(\d{2})\)',  # 유효기간(2025.08.20)
            r'유효기간\((\d{4})-(\d{2})-(\d{2})\)',   # 유효기간(2025-08-20)
            r'유효기간\((\d{4})(\d{2})(\d{2})\)',     # 유효기간(20250820)
            r'유효기간[_\s](\d{4})[\.\-](\d{2})[\.\-](\d{2})',  # 유효기간_2025.08.20
            r'(\d{4})[\.\-](\d{2})[\.\-](\d{2})[_\s]유효기간',  # 2025.08.20_유효기간
            r'제\d{4}-\d+호\([^)]+\)\s*유효기간\((\d{4})[\.\-](\d{2})[\.\-](\d{2})\)'  # 제2024-4806호(날짜) 유효기간(2025.08.20)
        ]
        
        # 각 패턴에 대해 개별 시도
        for i, pattern in enumerate(expiry_patterns):
            match = re.search(pattern, filename)
            if match:
                try:
                    groups = match.groups()
                    log_debug(f"패턴 {i+1} 매치: {pattern} -> {groups}")
                    
                    year, month, day = map(int, groups)
                    if 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                        date_str = f"{year}-{month:02d}-{day:02d}"
                        
                        # 오류 날짜 필터링
                        if date_str in ERROR_DATES:
                            log_debug(f"오류 날짜 감지: {date_str}")
                            continue
                        
                        # 성공한 패턴/날짜 로깅    
                        log_debug(f"유효기간 추출 성공: {date_str} (패턴 {i+1})")
                        expiry_date = date_str
                        break
                    else:
                        log_debug(f"유효하지 않은 날짜 값: {year}-{month}-{day}")
                except Exception as e:
                    log_debug(f"날짜 추출 오류: {pattern} -> {str(e)}")
                    continue
        
        # 추출 실패 시 확장 패턴 시도 (날짜 형식이 다른 경우)
        if expiry_date is None:
            log_debug("기본 패턴 매치 실패, 확장 패턴 시도")
            extended_patterns = [
                r'유효기간[:\s]*([12]\d{3})[년\.\-]?([01]?\d)[월\.\-]?([0-3]?\d)[일]?',  # 유효기간: 2025년6월3일
                r'([12]\d{3})년?[^0-9]+([01]?\d)월?[^0-9]+([0-3]?\d)일?[^0-9]*유효',  # 2025년 6월 3일 유효
                r'유효[^0-9]*([12]\d{3})년?[^0-9]+([01]?\d)월?[^0-9]+([0-3]?\d)일?'   # 유효 2025년 6월 3일
            ]
            
            for i, pattern in enumerate(extended_patterns):
                match = re.search(pattern, filename)
                if match:
                    try:
                        groups = match.groups()
                        log_debug(f"확장 패턴 {i+1} 매치: {pattern} -> {groups}")
                        
                        year, month, day = map(int, groups)
                        if 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31:
                            date_str = f"{year}-{month:02d}-{day:02d}"
                            
                            if date_str in ERROR_DATES:
                                log_debug(f"오류 날짜 감지: {date_str}")
                                continue
                                
                            log_debug(f"확장 패턴으로 유효기간 추출 성공: {date_str}")
                            expiry_date = date_str
                            break
                        else:
                            log_debug(f"유효하지 않은 날짜 값: {year}-{month}-{day}")
                    except Exception as e:
                        log_debug(f"확장 패턴 날짜 추출 오류: {str(e)}")
                        continue
        
        if expiry_date is None:
            log_debug(f"파일명에서 유효기간 추출 실패: {filename}")
        
        return expiry_date
    
    # 파일명에서 준법감시필 번호 추출 함수 강화
    def extract_compliance_number(filename):
        """파일명에서 준법감시필 번호 추출"""
        log_debug(f"준법감시필 번호 추출 시작: {filename}")
        
        compliance_patterns = [
            r'제(\d{4})-(\d+)호',  # 제2024-4806호
            r'준법감시[_\-\s]*(\d{4})[_\-\s]*(\d+)',  # 준법감시-2024-123
            r'심의필[_\-\s]*(\d{4})[_\-\s]*(\d+)',    # 심의필-2024-123
            r'준법[_\-\s]*(\d{4})[_\-\s]*(\d+)'       # 준법-2024-123
        ]
        
        for i, pattern in enumerate(compliance_patterns):
            match = re.search(pattern, filename)
            if match:
                try:
                    year, number = match.groups()
                    
                    # 형식에 따라 준법감시필 번호 생성
                    if pattern.startswith(r'제'):
                        compliance_number = f"제{year}-{number}호"
                    elif pattern.startswith(r'준법감시'):
                        compliance_number = f"준법감시-{year}-{number}"
                    elif pattern.startswith(r'심의필'):
                        compliance_number = f"심의필-{year}-{number}"
                    else:
                        compliance_number = f"준법-{year}-{number}"
                    
                    log_debug(f"준법감시필 번호 추출 성공: {compliance_number} (패턴 {i+1})")
                    return compliance_number
                except Exception as e:
                    log_debug(f"준법감시필 번호 추출 오류: {str(e)}")
                    continue
        
        log_debug(f"파일명에서 준법감시필 번호 추출 실패: {filename}")
        return None
    
    # AI 및 N-gram을 활용한 파일 분석 함수 - 강화된 버전
    def analyze_file(file_path):
        """파일 정보를 분석하여 결과 딕셔너리 반환 - 강화된 버전"""
        try:
            filename = file_path.name
            file_index = target_files.index(file_path) if file_path in target_files else -1
            
            # 파일 순번 로깅
            log_debug(f"{'='*50}")
            log_debug(f"파일 분석 시작 [{file_index+1}/{len(target_files)}]: {filename}")
            
            # 파일명에서 유효기간 추출
            expiry_date = extract_expiry_date(filename)
            
            # 파일명에서 준법감시필 번호 추출
            compliance_number = extract_compliance_number(filename)
            
            # 파일명에서 유효기간 상태 결정
            filename_days_to_expiry = None
            filename_status = '상태 불명'
            
            if expiry_date:
                today = datetime.now().date()
                try:
                    expiry_date_obj = datetime.strptime(expiry_date, '%Y-%m-%d').date()
                    filename_days_to_expiry = (expiry_date_obj - today).days
                    
                    if filename_days_to_expiry < 0:
                        filename_status = '만료됨'
                    elif filename_days_to_expiry <= 30:
                        filename_status = '30일 이내 만료'
                    else:
                        filename_status = '유효함'
                    
                    log_debug(f"유효기간 상태: {filename_status} (남은 일수: {filename_days_to_expiry})")
                except:
                    log_debug(f"유효기간 상태 계산 오류")
            
            # 파일 내용 확인 변수 초기화
            compliance_info = None
            content_date = None
            content_days_to_expiry = None
            content_status = '상태 불명'
            
            # 여기서 파일 확장자 변수를 정의 (이 부분이 누락됨)
            ext = file_path.suffix.lower()  # 수정: 확장자 변수 정의
            content = ""
            
            # 파일 크기 확인
            try:
                file_size = file_path.stat().st_size
                is_large_file = file_size > 200 * 1024 * 1024  # 20MB 초과는 대용량
                if is_large_file:
                    log_debug(f"대용량 파일 감지: {file_size/(1024*1024):.2f}MB - 내용 분석 스킵")
                    # 대용량 파일은 분석하지 않음
                    return {
                        '파일명': filename,
                        '상태': filename_status if expiry_date else '상태 불명',
                        '만료일': expiry_date or '알 수 없음',
                        '남은 일수': filename_days_to_expiry,
                        '파일명_준법감시필': compliance_number,
                        '파일명_유효기간': expiry_date,
                        '파일명_유효기간_상태': filename_status if expiry_date else '정보 없음',
                        '파일명_남은일수': filename_days_to_expiry,
                        '파일내_준법감시필_번호': None,
                        '파일내_유효기간_날짜': None,
                        '파일내_유효기간_상태': '정보 없음',
                        '파일내_남은일수': None,
                        '파일경로': str(file_path.parent),
                        '파일인덱스': file_index + 1
                    }
            except:
                log_debug(f"파일 크기 확인 오류")

            # 텍스트 파일 처리
            if ext == '.txt':
                try:
                    with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                        content = f.read(100000)  # 최대 10만자만 읽기
                except:
                    try:
                        with open(file_path, 'r', encoding='cp949', errors='replace') as f:
                            content = f.read(100000)
                    except:
                        pass
            
            # DOCX 파일 처리 - 개선된 버전
            elif ext == '.docx':
                try:
                    import docx
                    doc = docx.Document(file_path)
                    
                    # 모든 텍스트 추출 (단락, 테이블, 헤더/푸터)
                    all_text = []
                    
                    # 1. 단락 추출
                    for para in doc.paragraphs:
                        if para.text.strip():
                            all_text.append(para.text)
                    
                    # 2. 테이블 추출
                    for table in doc.tables:
                        for row in table.rows:
                            row_text = ' | '.join([cell.text.strip() for cell in row.cells if cell.text.strip()])
                            if row_text:
                                all_text.append(row_text)
                    
                    # 3. 헤더/푸터 추출 (가능한 경우)
                    try:
                        for section in doc.sections:
                            if section.header:
                                for p in section.header.paragraphs:
                                    if p.text.strip():
                                        all_text.append(p.text)
                            if section.footer:
                                for p in section.footer.paragraphs:
                                    if p.text.strip():
                                        all_text.append(p.text)
                    except:
                        pass
                    
                    content = "\n".join(all_text)
                    log_debug(f"DOCX 파일 {file_path.name}에서 {len(all_text)}개 텍스트 블록 추출됨")
                except Exception as e:
                    log_debug(f"DOCX 처리 오류({file_path.name}): {str(e)}")
            
            # Excel 파일 처리 - 개선된 버전
            elif ext in ['.xlsx', '.xls']:
                try:
                    import pandas as pd
                    import openpyxl
                    xl = pd.ExcelFile(file_path, engine='openpyxl')
                    sheet_names = xl.sheet_names[:5]  # 최대 5개 시트만 처리
                    
                    all_text = []
                    for sheet_name in sheet_names:
                        try:
                            sheet_df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl', nrows=1000)
                            # 데이터프레임을 문자열로 변환할 때 좀 더 철저히 처리
                            sheet_df = sheet_df.fillna('')  # NaN 값을 빈 문자열로 변환
                            
                            # 모든 데이터를 문자열화
                            for col in sheet_df.columns:
                                sheet_df[col] = sheet_df[col].astype(str)
                                sheet_df[col] = sheet_df[col].str.strip()
                            
                            # 행별로 데이터 추출
                            for _, row in sheet_df.iterrows():
                                # 빈 값 제외하고 결합
                                row_values = [val for val in row.values if val and val.strip()]
                                if row_values:
                                    all_text.append(' | '.join(row_values))
                        except Exception as sheet_e:
                            log_debug(f"Excel 시트 '{sheet_name}' 처리 오류: {str(sheet_e)}")
                            continue
                    
                    content = '\n'.join(all_text)
                    log_debug(f"Excel 파일 {file_path.name}에서 {len(all_text)}개 행 추출됨")
                except Exception as e:
                    log_debug(f"Excel 처리 오류({file_path.name}): {str(e)}")
            
            # PowerPoint 파일 처리 - 개선된 버전
            elif ext == '.pptx':
                try:
                    from pptx import Presentation
                    prs = Presentation(file_path)
                    slide_texts = []
                    
                    # 슬라이드별 처리
                    for i, slide in enumerate(prs.slides):
                        # 슬라이드 번호 추가
                        slide_texts.append(f"---슬라이드 {i+1}---")
                        
                        # 모든 도형에서 텍스트 추출
                        for shape in slide.shapes:
                            # 텍스트가 있는 도형
                            if hasattr(shape, "text") and shape.text.strip():
                                slide_texts.append(shape.text)
                            
                            # 테이블 처리
                            if hasattr(shape, "has_table") and shape.has_table:
                                try:
                                    for row in shape.table.rows:
                                        row_text = ' | '.join([cell.text.strip() for cell in row.cells if cell.text.strip()])
                                        if row_text:
                                            slide_texts.append(row_text)
                                except:
                                    pass
                    
                    content = "\n".join(slide_texts)
                    log_debug(f"PPTX 파일 {file_path.name}에서 {len(slide_texts)}개 텍스트 블록 추출됨")
                except Exception as e:
                    log_debug(f"PPTX 처리 오류({file_path.name}): {str(e)}")
            
            # HWP 파일 처리 - 개선된 버전
            elif ext == '.hwp':
                try:
                    import olefile
                    if olefile.isOleFile(str(file_path)):
                        hwp_content = []
                        
                        with olefile.OleFile(str(file_path)) as ole:
                            # 스트림 목록 확인
                            streams = ole.listdir()
                            log_debug(f"HWP 파일 {file_path.name} 스트림: {streams}")
                            
                            # PrvText 스트림 처리 (미리보기 텍스트)
                            if ole.exists('PrvText'):
                                try:
                                    prv_text = ole.openstream('PrvText')
                                    prv_bytes = prv_text.read()
                                    prv_text.close()
                                    
                                    # 여러 인코딩 시도
                                    for encoding in ['utf-16-le', 'cp949', 'euc-kr']:
                                        try:
                                            decoded = prv_bytes.decode(encoding, errors='replace')
                                            if decoded.strip():
                                                hwp_content.append(decoded)
                                                log_debug(f"HWP 미리보기 추출 성공({encoding}): {len(decoded)}자")
                                                break
                                        except:
                                            continue
                                except Exception as e:
                                    log_debug(f"HWP 미리보기 추출 오류: {str(e)}")
                            
                            # 문서 요약 정보 처리
                            if ole.exists('HwpSummaryInformation'):
                                try:
                                    summary = ole.openstream('HwpSummaryInformation')
                                    summary_bytes = summary.read()
                                    summary.close()
                                    
                                    # 가능한 인코딩으로 시도
                                    for encoding in ['utf-16-le', 'cp949', 'euc-kr']:
                                        try:
                                            # 인코딩 변환 후 유효한 문자만 필터링
                                            decoded = summary_bytes.decode(encoding, errors='replace')
                                            valid_text = ''.join(ch for ch in decoded if ch.isprintable())
                                            if valid_text.strip():
                                                hwp_content.append(valid_text)
                                                log_debug(f"HWP 요약정보 추출 성공({encoding}): {len(valid_text)}자")
                                                break
                                        except:
                                            continue
                                except Exception as e:
                                    log_debug(f"HWP 요약정보 추출 오류: {str(e)}")
                        
                        # 결과 합치기
                        content = "\n".join(hwp_content)
                        if content:
                            log_debug(f"HWP 파일 {file_path.name}에서 총 {len(content)}자 추출됨")
                    else:
                        log_debug(f"HWP 파일 {file_path.name}은 OLE 형식이 아님")
                except Exception as e:
                    log_debug(f"HWP 처리 오류({file_path.name}): {str(e)}")
            
            # PDF 파일 처리 - 새로 추가됨
            elif ext == '.pdf':
                try:
                    import PyPDF2
                    with open(file_path, 'rb') as file:
                        pdf_content = []
                        
                        try:
                            # 엄격하지 않은 모드로 PDF 읽기
                            reader = PyPDF2.PdfReader(file, strict=False)
                            
                            # 모든 페이지 처리
                            for page_num, page in enumerate(reader.pages):
                                try:
                                    page_text = page.extract_text()
                                    if page_text and page_text.strip():
                                        pdf_content.append(f"---페이지 {page_num+1}---")
                                        pdf_content.append(page_text)
                                        log_debug(f"PDF 페이지 {page_num+1} 추출 성공: {len(page_text)}자")
                                except Exception as page_e:
                                    log_debug(f"PDF 페이지 {page_num+1} 처리 오류: {str(page_e)}")
                                    continue
                            
                            content = "\n".join(pdf_content)
                            if len(pdf_content) > 0:
                                log_debug(f"PDF 파일 {file_path.name}에서 {len(pdf_content)}개 블록 추출됨")
                        except Exception as e:
                            log_debug(f"PDF 파일 구조 오류({file_path.name}): {str(e)}")
                except Exception as e:
                    log_debug(f"PDF 처리 오류({file_path.name}): {str(e)}")
            
            # 내용이 추출되었는지 확인하고 준법감시 키워드 확인
            if content:
                keywords = ['준법감시', '심의필', '준법감사', '유효기간', '만료일']
                found_keywords = [kw for kw in keywords if kw in content]

                if found_keywords:
                    log_debug(f"파일 {file_path.name}에서 키워드 발견: {', '.join(found_keywords)}")
                else:
                    log_debug(f"파일 {file_path.name}에서 키워드 미발견 (내용 길이: {len(content)}자)")
            
            # AI 및 N-gram을 활용한 분석
            if content:
                # AI 추출기 인스턴스 생성
                extractor = AIEnhancedComplianceExtractor(
                    api_key="yourkey",  # 실제 API 키로 교체
                    use_ngram=True,
                    use_ai=False  # API 키가 없으면 AI 사용 안함
                )
                
                # 문서 처리
                result = extractor.process_document(content)
                
                # 결과 추출
                compliance_info = result.get("compliance_number")
                content_date = result.get("expiry_date")
                
                # 오류 날짜 필터링
                if content_date in ERROR_DATES:
                    content_date = None
                
                # 유효기간 상태 계산
                if content_date:
                    today = datetime.now().date()
                    try:
                        content_date_obj = datetime.strptime(content_date, '%Y-%m-%d').date()
                        content_days_to_expiry = (content_date_obj - today).days
                        
                        if content_days_to_expiry < 0:
                            content_status = '만료됨'
                        elif content_days_to_expiry <= 30:
                            content_status = '30일 이내 만료'
                        else:
                            content_status = '유효함'
                    except:
                        pass
            
            # 정보 추출 로직 (파일명 우선, 내용 차선)
            final_date = expiry_date or content_date
            final_days_to_expiry = filename_days_to_expiry or content_days_to_expiry
            final_status = filename_status if expiry_date else (content_status if content_date else '상태 불명')
            
            # 추가 필터링: 최종 날짜가 오류 날짜인 경우
            if final_date in ERROR_DATES:
                log_debug(f"최종 날짜가 오류 날짜임: {final_date}")
                final_date = None
                final_days_to_expiry = None
                final_status = '상태 불명'
            
            # 분석 결과 요약 로깅
            log_debug(f"분석 결과: 상태={final_status}, 만료일={final_date}, 준법번호={compliance_number}")
            
            # 결과 딕셔너리 반환
            return {
                '파일명': filename,
                '상태': final_status,
                '만료일': final_date or '알 수 없음',
                '남은 일수': final_days_to_expiry,
                
                # 파일명 정보
                '파일명_준법감시필': compliance_number,
                '파일명_유효기간': expiry_date,
                '파일명_유효기간_상태': filename_status if expiry_date else '정보 없음',
                '파일명_남은일수': filename_days_to_expiry,
                
                # 파일 내용 정보
                '파일내_준법감시필_번호': compliance_info,
                '파일내_유효기간_날짜': content_date,
                '파일내_유효기간_상태': content_status if content_date else '정보 없음',
                '파일내_남은일수': content_days_to_expiry,
                
                '파일경로': str(file_path.parent),
                '파일인덱스': file_index + 1  # 파일 인덱스 추가 (문제 파일 추적용)
            }
            
        except Exception as e:
            log_debug(f"파일 분석 중 오류: {str(e)}")
            log_debug(traceback.format_exc())
            # 오류 발생 시 기본 정보만 반환
            return {
                '파일명': file_path.name,
                '상태': '상태 불명',
                '만료일': '알 수 없음',
                '남은 일수': None,
                '파일명_준법감시필': None, 
                '파일명_유효기간': None,
                '파일명_유효기간_상태': '정보 없음',
                '파일명_남은일수': None,
                '파일내_준법감시필_번호': None,
                '파일내_유효기간_날짜': None,
                '파일내_유효기간_상태': '정보 없음',
                '파일내_남은일수': None,
                '파일경로': str(file_path.parent),
                '파일인덱스': target_files.index(file_path) + 1 if file_path in target_files else -1
            }

    # 오류 날짜 목록 - 이 날짜들은 검사에서 제외됩니다
    ERROR_DATES = ['2024-11-30', '2020-12-31']

    # 파일 확장자 목록 정의 (추가된 코드)
    file_extensions = ['.docx', '.xlsx', '.pptx', '.hwp', '.txt', '.pdf', '.xls', '.doc', '.ppt']

    # 디버깅 모드 활성화
    DEBUG_MODE = True
    DEBUG_FILE = "debug_log.txt"

    # 병렬 파일 검색 - 간소화 버전
    def search_files_in_directory(directory):
        found_files = []
        # 전역 변수 file_extensions 사용
        for ext in file_extensions:
            try:
                for file_path in Path(directory).glob(f'**/*{ext}'):
                    # 키워드 필터링 제거 - 모든 파일 추가
                    found_files.append(file_path)
            except Exception as e:
                print(f"디렉토리 {directory} 검색 중 오류: {e}")
        return found_files

    # 디렉토리 목록 생성
    all_dirs = []
    try:
        for d in base_path.iterdir():
            if d.is_dir():
                all_dirs.append(d)
    except Exception as e:
        print(f"디렉토리 목록 생성 중 오류: {e}")

    if not all_dirs:  # 하위 디렉토리가 없으면 현재 디렉토리 검색
        all_dirs = [base_path]

    print(f"총 {len(all_dirs)}개 디렉토리 검색 예정")

    # 파일 검색 (병렬 처리 단순화)
    target_files = collect_target_files(BASE_PATH)
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        search_results = list(executor.map(search_files_in_directory, all_dirs))
        for result in search_results:
            target_files.extend(result)

    print(f"총 {len(target_files)}개 파일을 찾았습니다. ({time.time() - start_time:.2f}초)")

    # 파일 분석 (배치 처리 방식으로 변경)
    print("파일 분석 시작...")
    data = []

    # 파일을 배치로 나누어 처리
    batch_size = 1000  # 한 번에 처리할 파일 수
    total_batches = (len(target_files) + batch_size - 1) // batch_size
    print(f"총 {total_batches}개 배치로 나누어 처리합니다.")

    # 배치별 처리 및 중간 결과 저장
    for batch_idx in range(total_batches):
        batch_start = batch_idx * batch_size
        batch_end = min((batch_idx + 1) * batch_size, len(target_files))
        batch_files = target_files[batch_start:batch_end]
        
        print(f"\n배치 {batch_idx + 1}/{total_batches} 처리 중 ({batch_end - batch_start}개 파일)...")
        
        # 현재 배치 병렬 처리
        batch_data = []
        max_workers = min(os.cpu_count() or 4, 8)  # 최대 8개 워커
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # 각 파일별로 분석 작업 예약
            future_to_file = {executor.submit(analyze_file, file_path): file_path for file_path in batch_files}
            
            # 완료된 작업 처리
            for future in tqdm(concurrent.futures.as_completed(future_to_file), total=len(batch_files), desc=f"배치 {batch_idx + 1} 분석 중"):
                try:
                    result = future.result()
                    if result:
                        batch_data.append(result)
                except Exception as e:
                    file_path = future_to_file[future]
                    print(f"\n파일 {file_path.name} 처리 중 오류: {e}")
        
        # 현재 배치 결과 저장
        data.extend(batch_data)
        print(f"배치 {batch_idx + 1} 완료: {len(batch_data)}개 파일 처리됨")
        
        # 메모리 정리
        gc.collect()
        
        # 진행 상황 저장 (선택 사항)
        #if batch_idx % 5 == 4 or batch_idx == total_batches - 1:  # 5개 배치마다 또는 마지막 배치
        #    try:
        #        interim_df = pd.DataFrame(data)
        #        interim_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        #        interim_file = f"interim_results_{interim_timestamp}.csv"
        #        interim_df.to_csv(interim_file, index=False, encoding='utf-8-sig')
        #        print(f"중간 결과 저장됨: {interim_file}")
        #    except Exception as e:
        #        print(f"중간 결과 저장 중 오류: {e}")

    print(f"총 {len(data)}개 파일 분석 완료")
    
    # 데이터프레임 생성
    if data:
        df = pd.DataFrame(data)
        
        # 유효기간 날짜 검증 및 오류 수정 - 강화된 검증
        def validate_date(date_str):
            """유효기간 날짜 검증 함수"""
            if not isinstance(date_str, str) or date_str == '알 수 없음':
                return date_str
                
            # 오류 날짜 명시적으로 제거
            if date_str in ERROR_DATES:
                return None
                
            try:
                # 날짜 형식 검증
                date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                
                # 추가 유효성 검증 (너무 과거나 먼 미래의 날짜는 의심)
                today = datetime.now()
                five_years_ago = today.replace(year=today.year - 5)
                five_years_future = today.replace(year=today.year + 5)
                
                if date_obj < five_years_ago or date_obj > five_years_future:
                    # 너무 이상한 날짜는 로그로 기록
                    print(f"의심스러운 날짜 검출: {date_str} (허용 범위: {five_years_ago.strftime('%Y-%m-%d')} ~ {five_years_future.strftime('%Y-%m-%d')})")
                
                return date_str
            except:
                return None
        
        # 모든 날짜 컬럼에 대한 검증 적용
        for col in ['만료일', '파일명_유효기간', '파일내_유효기간_날짜']:
            if col in df.columns:
                df[col] = df[col].apply(validate_date)
        
        # 유효기간 날짜가 None인 경우 상태도 '상태 불명'으로 변경
        df.loc[df['만료일'].isna() | (df['만료일'] == '알 수 없음'), '상태'] = '상태 불명'
        
        # 추가 확인: 오류 날짜를 포함하는 행 찾기
        for error_date in ERROR_DATES:
            problem_files = df[df.apply(lambda row: any(str(val) == error_date for val in row), axis=1)]
            if not problem_files.empty:
                print(f"\n경고: {error_date} 날짜가 여전히 {len(problem_files)}개 파일에서 발견됨")
                print("해당 파일:")
                for idx, row in problem_files.iterrows():
                    print(f"- {row['파일명']}")
        
        # 결과 열 정리 - 불필요한 컬럼 제거
        column_order = [
            '파일명', '상태', '만료일', '남은 일수', 
            '파일명_준법감시필', '파일명_유효기간', '파일명_유효기간_상태', '파일명_남은일수',
            '파일내_준법감시필_번호',
            '파일내_유효기간_날짜', '파일내_유효기간_상태', '파일내_남은일수',
            '파일경로'
        ]
        
        # 존재하는 열만 선택
        existing_columns = [col for col in column_order if col in df.columns]
        df = df[existing_columns]
        
        # 결과 정렬 (상태별)
        status_order = {'만료됨': 0, '30일 이내 만료': 1, '유효함': 2, '상태 불명': 3}
        df['status_order'] = df['상태'].map(lambda x: status_order.get(x, 4))
        df = df.sort_values(by=['status_order', '남은 일수'])
        df = df.drop(columns=['status_order'])
        
        # 현재 디렉토리에 저장
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"ibk_준법감시_유효기간_{timestamp}.csv"
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"결과 파일이 현재 디렉토리에 저장되었습니다: {output_file}")
        
        # Excel 파일로 저장하고 열 너비 자동 조정
        excel_file = f"ibk_준법감시_유효기간_{timestamp}.xlsx"

        # ExcelWriter 사용
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            # DataFrame을 Excel로 변환
            df.to_excel(writer, index=False, sheet_name='준법감시')
            
            # 워크북과 워크시트 가져오기
            workbook = writer.book
            worksheet = writer.sheets['준법감시']
            
            # 열 너비 자동 조정
            for i, column in enumerate(df.columns):
                # 열 이름의 길이 확인
                column_width = max(len(str(column)), 10)  # 최소 너비 10
                
                # 열의 데이터 중 가장 긴 값의 길이 확인 (최대 100행까지만 확인)
                max_length = 0
                for j in range(min(len(df), 100)):  # 처리 시간 단축을 위해 최대 100행까지만 확인
                    cell_value = str(df.iloc[j, i])
                    max_length = max(max_length, len(cell_value))
                
                # 최종 열 너비 결정 (최대 100자, 기본 폰트 기준)
                column_width = min(max(column_width, max_length + 2), 100)  # +2는 여유 공간
                
                # 열 너비 설정
                col_letter = openpyxl.utils.get_column_letter(i + 1)
                worksheet.column_dimensions[col_letter].width = column_width
            
            # 헤더 행 스타일 설정
            header_font = openpyxl.styles.Font(bold=True)
            header_fill = openpyxl.styles.PatternFill(start_color='E6E6E6', end_color='E6E6E6', fill_type='solid')
            
            for cell in worksheet[1]:
                cell.font = header_font
                cell.fill = header_fill
            
            # 데이터 행에 조건부 서식 설정 (상태에 따른 색상)
            status_colors = {
                '만료됨': 'FFC7CE',  # 밝은 빨강
                '30일 이내 만료': 'FFEB9C',  # 밝은 노랑
                '유효함': 'C6EFCE',  # 밝은 녹색
                '상태 불명': 'DDDDDD'  # 회색
            }
            
            # 상태 열 인덱스 찾기
            status_col_idx = df.columns.get_loc('상태') + 1  # Excel은 1부터 시작
            status_col_letter = openpyxl.utils.get_column_letter(status_col_idx)
            
            # 각 상태별로 조건부 서식 설정
            for status, color in status_colors.items():
                rule = openpyxl.formatting.rule.CellIsRule(
                    operator='equal',
                    formula=[f'"{status}"'],
                    stopIfTrue=True,
                    fill=openpyxl.styles.PatternFill(start_color=color, end_color=color, fill_type='solid')
                )
                
                # 조건부 서식 적용 범위 (상태 열만)
                cell_range = f'{status_col_letter}2:{status_col_letter}{len(df) + 1}'
                worksheet.conditional_formatting.add(cell_range, rule)

        print(f"Excel 파일이 자동 열 너비 조정과 서식을 적용하여 저장되었습니다: {excel_file}")
        
        # 총 소요 시간
        total_time = time.time() - start_time
        print(f"총 실행 시간: {total_time:.2f}초 (파일당 평균: {total_time/len(target_files):.2f}초)")
        
        # 결과 통계
        status_counts = df['상태'].value_counts().to_dict()
        print("\n=== 분석 결과 통계 ===")
        print(f"- 만료됨: {status_counts.get('만료됨', 0)}개")
        print(f"- 30일 이내 만료: {status_counts.get('30일 이내 만료', 0)}개")
        print(f"- 유효함: {status_counts.get('유효함', 0)}개")
        print(f"- 상태 불명: {status_counts.get('상태 불명', 0)}개")
        print(f"- 총 파일 수: {len(df)}개")
        
        # 정보 출처 통계
        source_stats = {
            '파일명에서만 발견': len(df[(df['파일명_유효기간'].notna()) & (df['파일내_유효기간_날짜'].isna())]),
            '파일 내용에서만 발견': len(df[(df['파일명_유효기간'].isna()) & (df['파일내_유효기간_날짜'].notna())]),
            '파일명과 내용 모두에서 발견': len(df[(df['파일명_유효기간'].notna()) & (df['파일내_유효기간_날짜'].notna())]),
            '어디에서도 발견되지 않음': len(df[(df['파일명_유효기간'].isna()) & (df['파일내_유효기간_날짜'].isna())])
        }
        
        print("\n=== 정보 출처 통계 ===")
        for source, count in source_stats.items():
            print(f"- {source}: {count}개 ({count/len(df)*100:.1f}%)")
        
        # 결과 미리보기
        print("\n === 결과 데이터프레임 미리보기 ===")
        display(df.head())
    
    else:
        print("분석할 파일이 없거나 모든 파일 처리 중 오류가 발생했습니다.")

except Exception as e:
    print(f"오류 발생: {e}")
    print(traceback.format_exc())